# Plastic Map

## Imports

In [3]:
from modules import dart_files, tiff_files, rsdata_classification, rsdata_charts
from scipy.stats import ks_2samp
import os
import pandas as pd

## Parameters

In [4]:
feature_names = ['Blue', 'Green', 'Red', 'RedEdge1', 'RedEdge2', 'RedEdge3', 'NIR1', 'NIR2', 'SWIR1', 'SWIR2']
radiometric_indexes = ['NDWI', 'WRI', 'NDVI', 'AWEI', 'MNDWI', 'SR', 'PI', 'RNDVI', 'FDI'] #'NDMI' excluído porque é identico ao NDWI 

## Simulated dataset (DART)

In [3]:
path, polymers, cover_percent = dart_files.open_folders(str(input("Simulated files path: ")))
#Folders need to be structured like path/polymers/coverage percent/files.asc 
#For example: files/dart_files/1200x1200

Dart files path: files/dart_files/1200x1200


### Resampling

#### Resampling 20 meter bands by nearest neighbor

In [4]:
nn_data = dart_files.get_images(path, polymers, cover_percent, "nearest") #resample_method refere-se às opções

As bandas da imagem  LDPE/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da 

In [5]:
for image in nn_data:
    image.setAreaLabel(0, (int(image.getXSize() / 2) - 1), 0, (image.getYSize() - 1), "Water") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setAreaLabel(int(image.getXSize() / 2), (image.getXSize() - 1), 0, (image.getYSize() - 1), "Sand") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setGridLabel(29, 89, 4, 27, 87, 4, "Plastic")
    for pixel in image.getPixels():
        pixel.setLabel(image.getLabelsMap())
        pixel.setCoverPercent("Plastic", image.getPlasticCoverPercent())

In [6]:
dart_nn = dart_files.build_dataset(nn_data)

In [9]:
os.chdir('../')
os.chdir('../')
os.chdir('../')

In [10]:
dart_nn['Polymer'] = [x.split('/')[0] for x in dart_nn['Path']]
dart_nn

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,Red,SWIR1,SWIR2,Cover_percent,Label,Polymer
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC


In [11]:
dart_nn.to_csv(str(input("Path/filename: ")))
#For example: files/csv_files/dataset_dart_nn.csv

Path/filename: files/csv_files/dataset_dart_nn.csv


#### Resampling 20 meter bands by bilinear interpolation

In [12]:
os.chdir(path)
bilinear_data = dart_files.get_images(path, polymers, cover_percent, "bilinear") #resample_method refere-se às opções

As bandas da imagem  LDPE/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da 

In [13]:
for image in bilinear_data:
    image.setAreaLabel(0, (int(image.getXSize() / 2) - 1), 0, (image.getYSize() - 1), "Water") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setAreaLabel(int(image.getXSize() / 2), (image.getXSize() - 1), 0, (image.getYSize() - 1), "Sand") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setGridLabel(29, 89, 4, 27, 87, 4, "Plastic")
    for pixel in image.getPixels():
        pixel.setLabel(image.getLabelsMap())
        pixel.setCoverPercent("Plastic", image.getPlasticCoverPercent())

In [14]:
dart_bilinear = dart_files.build_dataset(bilinear_data)

In [15]:
os.chdir('../')
os.chdir('../')
os.chdir('../')

In [16]:
dart_bilinear['Polymer'] = [x.split('/')[0] for x in dart_bilinear['Path']]
dart_bilinear

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,Red,SWIR1,SWIR2,Cover_percent,Label,Polymer
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC


In [17]:
dart_bilinear.to_csv(str(input("Path/filename: ")))
#For example: files/csv_files/dataset_dart_bilinear.csv

Path/filename: files/csv_files/dataset_dart_bilinear.csv


#### Resampling 20 meter bands by cubic interpolation

In [18]:
os.chdir(path)
cubic_data = dart_files.get_images(path, polymers, cover_percent, "cubic")

As bandas da imagem  LDPE/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  LDPE/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/100/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/20/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/40/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/60/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da imagem  MicroNapo/80/  serão reamostradas para a melhor resolução disponível (120, 120)
As bandas da 

In [19]:
for image in cubic_data:
    image.setAreaLabel(0, (int(image.getXSize() / 2) - 1), 0, (image.getYSize() - 1), "Water") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setAreaLabel(int(image.getXSize() / 2), (image.getXSize() - 1), 0, (image.getYSize() - 1), "Sand") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    image.setGridLabel(29, 89, 4, 27, 87, 4, "Plastic")
    for pixel in image.getPixels():
        pixel.setLabel(image.getLabelsMap())
        pixel.setCoverPercent("Plastic", image.getPlasticCoverPercent())

In [20]:
dart_cubic = dart_files.build_dataset(cubic_data)

In [21]:
os.chdir('../')
os.chdir('../')
os.chdir('../')

In [22]:
dart_cubic['Polymer'] = [x.split('/')[0] for x in dart_cubic['Path']]
dart_cubic

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,Red,SWIR1,SWIR2,Cover_percent,Label,Polymer
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,0.0245,0.0186,0.0163,0,Water,LDPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,0.7842,0.5262,0.1232,0,Sand,PVC


In [23]:
dart_cubic.to_csv(str(input("Path/filename: ")))
#For example: files/csv_files/dataset_dart_cubic.csv

Path/filename: files/csv_files/dataset_dart_cubic.csv


### Building data frames

#### Building data frames

##### Default dataset (Bilinear interpolation) 

Adding labels translation

In [6]:
dataset_dart = pd.read_csv(str(input("Path/filename: "))) #For example: files/csv_files/dataset_dart_bilinear.csv
dataset_dart.drop('Unnamed: 0', axis=1, inplace=True)

dart_label = []
for i in range(len(dataset_dart)):
    if dataset_dart.at[i, 'Label'] == 'Water':
        dart_label.append('Água')
    elif dataset_dart.at[i, 'Label'] == 'Sand':
        dart_label.append('Areia')
    elif dataset_dart.at[i, 'Label'] == 'Plastic':
        dart_label.append('Plástico')

dataset_dart['Classe'] = dart_label

Path/filename: files/csv_files/dataset_dart_bilinear.csv


Adding radiometric indices

In [7]:
dataset_dart['NDWI'] = (dataset_dart['Green'] - dataset_dart['NIR1']) / (dataset_dart['Green'] + dataset_dart['NIR1'])
dataset_dart['WRI'] = (dataset_dart['Green'] + dataset_dart['Red']) / (dataset_dart['NIR1'] + dataset_dart['SWIR2'])
dataset_dart['NDVI'] = (dataset_dart['NIR1'] - dataset_dart['Red']) / (dataset_dart['NIR1'] + dataset_dart['Red'])
dataset_dart['AWEI'] = 4 * (dataset_dart['Green'] - dataset_dart['SWIR2']) - (0.25 * dataset_dart['NIR1'] + 2.75 * dataset_dart['SWIR1'])
dataset_dart['MNDWI'] = (dataset_dart['Green'] - dataset_dart['SWIR2']) / (dataset_dart['Green'] + dataset_dart['SWIR2'])
dataset_dart['SR'] = dataset_dart['NIR1'] / dataset_dart['Red']
dataset_dart['PI'] = dataset_dart['NIR1'] / (dataset_dart['NIR1'] + dataset_dart['Red'])
dataset_dart['RNDVI'] = (dataset_dart['Red'] - dataset_dart['NIR1']) / (dataset_dart['Red'] + dataset_dart['NIR1'])
dataset_dart['FDI'] = dataset_dart['NIR1'] - (dataset_dart['RedEdge2'] + (dataset_dart['SWIR1'] - dataset_dart['RedEdge2']) * ((dataset_dart['NIR1'] - dataset_dart['Red']) / (dataset_dart['SWIR1'] - dataset_dart['Red'])) * 10)

dataset_dart

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,...,Classe,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892


Data cleaning

In [8]:
dividers_dart = dict()
dividers_dart.update({"Green + NIR1": dataset_dart['Green'] + dataset_dart['NIR1']})
dividers_dart.update({"SWIR2 + NIR1": dataset_dart['NIR1'] + dataset_dart['SWIR2']})
dividers_dart.update({"Red + NIR1": dataset_dart['NIR1'] + dataset_dart['Red']})
dividers_dart.update({"0.25 * NIR1 + 2.75 * SWIR1": 0.25 * dataset_dart['NIR1'] + 2.75 * dataset_dart['SWIR1']})
dividers_dart.update({"Swir2 + * Green": dataset_dart['Green'] + dataset_dart['SWIR2']})
dividers_dart.update({"Red": dataset_dart['Red']})
dividers_dart.update({"SWIR1 - Red": dataset_dart['SWIR1'] - dataset_dart['Red']})

zeros_dart = dict()

for key in dividers_dart.keys():
    i = 0
    for value in dividers_dart[key]:
        if value == 0:
            i += 1
    zeros_dart.update({key:i})
zeros_dart

{'Green + NIR1': 0,
 'SWIR2 + NIR1': 0,
 'Red + NIR1': 0,
 '0.25 * NIR1 + 2.75 * SWIR1': 0,
 'Swir2 + * Green': 0,
 'Red': 0,
 'SWIR1 - Red': 512}

In [9]:
indexes = dataset_dart.query('FDI < -1000').index #deletando amostras com valor -inf derivados da divisâo por zero no FDI
dataset_dart.drop(indexes,  axis=0, inplace=True)
dataset_dart

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,...,Classe,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892


##### Other datasets (Nearest neighbor and Cubic interpolation) 

Adding labels translation and radiometric indices and cleaning data 

In [10]:
dataset_dart_cubic = pd.read_csv(str(input("Path/filename: "))) #For example: files/csv_files/dataset_dart_cubic.csv
dataset_dart_cubic.drop('Unnamed: 0', axis=1, inplace=True)

dart_label = []
for i in range(len(dataset_dart_cubic)):
    if dataset_dart_cubic.at[i, 'Label'] == 'Water':
        dart_label.append('Água')
    elif dataset_dart_cubic.at[i, 'Label'] == 'Sand':
        dart_label.append('Areia')
    elif dataset_dart_cubic.at[i, 'Label'] == 'Plastic':
        dart_label.append('Plástico')

dataset_dart_cubic['Classe'] = dart_label

Path/filename: files/csv_files/dataset_dart_cubic.csv


In [11]:
dataset_dart_cubic['NDWI'] = (dataset_dart_cubic['Green'] - dataset_dart_cubic['NIR1']) / (dataset_dart_cubic['Green'] + dataset_dart_cubic['NIR1'])
dataset_dart_cubic['WRI'] = (dataset_dart_cubic['Green'] + dataset_dart_cubic['Red']) / (dataset_dart_cubic['NIR1'] + dataset_dart_cubic['SWIR2'])
dataset_dart_cubic['NDVI'] = (dataset_dart_cubic['NIR1'] - dataset_dart_cubic['Red']) / (dataset_dart_cubic['NIR1'] + dataset_dart_cubic['Red'])
dataset_dart_cubic['AWEI'] = 4 * (dataset_dart_cubic['Green'] - dataset_dart_cubic['SWIR2']) - (0.25 * dataset_dart_cubic['NIR1'] + 2.75 * dataset_dart_cubic['SWIR1'])
dataset_dart_cubic['MNDWI'] = (dataset_dart_cubic['Green'] - dataset_dart_cubic['SWIR2']) / (dataset_dart_cubic['Green'] + dataset_dart_cubic['SWIR2'])
dataset_dart_cubic['SR'] = dataset_dart_cubic['NIR1'] / dataset_dart_cubic['Red']
dataset_dart_cubic['PI'] = dataset_dart_cubic['NIR1'] / (dataset_dart_cubic['NIR1'] + dataset_dart_cubic['Red'])
dataset_dart_cubic['RNDVI'] = (dataset_dart_cubic['Red'] - dataset_dart_cubic['NIR1']) / (dataset_dart_cubic['Red'] + dataset_dart_cubic['NIR1'])
dataset_dart_cubic['FDI'] = dataset_dart_cubic['NIR1'] - (dataset_dart_cubic['RedEdge2'] + (dataset_dart_cubic['SWIR1'] - dataset_dart_cubic['RedEdge2']) * ((dataset_dart_cubic['NIR1'] - dataset_dart_cubic['Red']) / (dataset_dart_cubic['SWIR1'] - dataset_dart_cubic['Red'])) * 10)

dataset_dart_cubic

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,...,Classe,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892


In [12]:
dividers_dart_cubic = dict()
dividers_dart_cubic.update({"Green + NIR1": dataset_dart_cubic['Green'] + dataset_dart_cubic['NIR1']})
dividers_dart_cubic.update({"SWIR2 + NIR1": dataset_dart_cubic['NIR1'] + dataset_dart_cubic['SWIR2']})
dividers_dart_cubic.update({"Red + NIR1": dataset_dart_cubic['NIR1'] + dataset_dart_cubic['Red']})
dividers_dart_cubic.update({"0.25 * NIR1 + 2.75 * SWIR1": 0.25 * dataset_dart_cubic['NIR1'] + 2.75 * dataset_dart_cubic['SWIR1']})
dividers_dart_cubic.update({"Swir2 + * Green": dataset_dart_cubic['Green'] + dataset_dart_cubic['SWIR2']})
dividers_dart_cubic.update({"Red": dataset_dart_cubic['Red']})
dividers_dart_cubic.update({"SWIR1 - Red": dataset_dart_cubic['SWIR1'] - dataset_dart_cubic['Red']})

zeros_dart_cubic = dict()

for key in dividers_dart_cubic.keys():
    i = 0
    for value in dividers_dart_cubic[key]:
        if value == 0:
            i += 1
    zeros_dart_cubic.update({key:i})
zeros_dart_cubic

{'Green + NIR1': 0,
 'SWIR2 + NIR1': 0,
 'Red + NIR1': 0,
 '0.25 * NIR1 + 2.75 * SWIR1': 0,
 'Swir2 + * Green': 0,
 'Red': 0,
 'SWIR1 - Red': 60}

In [13]:
indexes = dataset_dart_cubic.query('FDI < -1000').index #deletando amostras com valor -inf derivados da divisâo por zero no FDI
dataset_dart_cubic.drop(indexes,  axis=0, inplace=True)
dataset_dart_cubic

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,...,Classe,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892


In [14]:
dataset_dart_nn = pd.read_csv(str(input("Path/filename: ")))#For example: files/csv_files/dataset_dart_nn.csv
dataset_dart_nn.drop('Unnamed: 0', axis=1, inplace=True)

dart_label = []
for i in range(len(dataset_dart_nn)):
    if dataset_dart_nn.at[i, 'Label'] == 'Water':
        dart_label.append('Água')
    elif dataset_dart_nn.at[i, 'Label'] == 'Sand':
        dart_label.append('Areia')
    elif dataset_dart_nn.at[i, 'Label'] == 'Plastic':
        dart_label.append('Plástico')

dataset_dart_nn['Classe'] = dart_label

Path/filename: files/csv_files/dataset_dart_nn.csv


In [15]:
dataset_dart_nn['NDWI'] = (dataset_dart_nn['Green'] - dataset_dart_nn['NIR1']) / (dataset_dart_nn['Green'] + dataset_dart_nn['NIR1'])
dataset_dart_nn['WRI'] = (dataset_dart_nn['Green'] + dataset_dart_nn['Red']) / (dataset_dart_nn['NIR1'] + dataset_dart_nn['SWIR2'])
dataset_dart_nn['NDVI'] = (dataset_dart_nn['NIR1'] - dataset_dart_nn['Red']) / (dataset_dart_nn['NIR1'] + dataset_dart_nn['Red'])
dataset_dart_nn['AWEI'] = 4 * (dataset_dart_nn['Green'] - dataset_dart_nn['SWIR2']) - (0.25 * dataset_dart_nn['NIR1'] + 2.75 * dataset_dart_nn['SWIR1'])
dataset_dart_nn['MNDWI'] = (dataset_dart_nn['Green'] - dataset_dart_nn['SWIR2']) / (dataset_dart_nn['Green'] + dataset_dart_nn['SWIR2'])
dataset_dart_nn['SR'] = dataset_dart_nn['NIR1'] / dataset_dart_nn['Red']
dataset_dart_nn['PI'] = dataset_dart_nn['NIR1'] / (dataset_dart_nn['NIR1'] + dataset_dart_nn['Red'])
dataset_dart_nn['RNDVI'] = (dataset_dart_nn['Red'] - dataset_dart_nn['NIR1']) / (dataset_dart_nn['Red'] + dataset_dart_nn['NIR1'])
dataset_dart_nn['FDI'] = dataset_dart_nn['NIR1'] - (dataset_dart_nn['RedEdge2'] + (dataset_dart_nn['SWIR1'] - dataset_dart_nn['RedEdge2']) * ((dataset_dart_nn['NIR1'] - dataset_dart_nn['Red']) / (dataset_dart_nn['SWIR1'] - dataset_dart_nn['Red'])) * 10)

dataset_dart_nn

,Path,Line,Column,Blue,Green,NIR1,NIR2,RedEdge1,RedEdge2,RedEdge3,...,Classe,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,LDPE/100/,0,0,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
1,LDPE/100/,0,1,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
2,LDPE/100/,0,2,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
3,LDPE/100/,0,3,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
4,LDPE/100/,0,4,0.0324,0.0386,0.0198,0.0198,0.0236,0.0208,0.0199,...,Água,0.321918,1.747922,-0.106095,0.033100,0.406193,0.808163,0.446953,0.106095,0.016525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,PVC/80/,119,115,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431996,PVC/80/,119,116,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431997,PVC/80/,119,117,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892
431998,PVC/80/,119,118,0.6315,0.7185,0.8073,0.8031,0.7965,0.8053,0.8135,...,Areia,-0.058199,1.614938,0.014515,0.732325,0.707259,1.029457,0.507257,-0.014515,-0.247892


In [16]:
dividers_dart_nn = dict()
dividers_dart_nn.update({"Green + NIR1": dataset_dart_nn['Green'] + dataset_dart_nn['NIR1']})
dividers_dart_nn.update({"SWIR2 + NIR1": dataset_dart_nn['NIR1'] + dataset_dart_nn['SWIR2']})
dividers_dart_nn.update({"Red + NIR1": dataset_dart_nn['NIR1'] + dataset_dart_nn['Red']})
dividers_dart_nn.update({"0.25 * NIR1 + 2.75 * SWIR1": 0.25 * dataset_dart_nn['NIR1'] + 2.75 * dataset_dart_nn['SWIR1']})
dividers_dart_nn.update({"Swir2 + * Green": dataset_dart_nn['Green'] + dataset_dart_nn['SWIR2']})
dividers_dart_nn.update({"Red": dataset_dart_nn['Red']})
dividers_dart_nn.update({"SWIR1 - Red": dataset_dart_nn['SWIR1'] - dataset_dart_nn['Red']})

zeros_dart_nn = dict()

for key in dividers_dart_nn.keys():
    i = 0
    for value in dividers_dart_nn[key]:
        if value == 0:
            i += 1
    zeros_dart_nn.update({key:i})
zeros_dart_nn

{'Green + NIR1': 0,
 'SWIR2 + NIR1': 0,
 'Red + NIR1': 0,
 '0.25 * NIR1 + 2.75 * SWIR1': 0,
 'Swir2 + * Green': 0,
 'Red': 0,
 'SWIR1 - Red': 0}

#### Building subdataframes

In [17]:
dart_subdatasets = dict()
dart_subdatasets['plastic'] = dataset_dart.loc[dataset_dart['Label'] == "Plastic"].copy()
dart_subdatasets['water'] = dataset_dart.loc[dataset_dart['Label'] == "Water"].copy()
dart_subdatasets['sand'] = dataset_dart.loc[dataset_dart['Label'] == "Sand"].copy()
dart_subdatasets['plastic_and_water'] = dataset_dart.query("Label == 'Water' or Label == 'Plastic'").copy()

In [18]:
dart_plastic_in_water, dart_plastic_in_sand = [], []

for i in dart_subdatasets['plastic'].index:
    if dataset_dart.at[i - 1, 'Label'] == 'Water':
        dart_plastic_in_water.append(dart_subdatasets['plastic'].loc[i])
    elif dataset_dart.at[i - 1, 'Label'] == 'Sand':
        dart_plastic_in_sand.append(dart_subdatasets['plastic'].loc[i])
    else:
        print("Erro")
        
dart_subdatasets['plastic_in_water'], dart_subdatasets['plastic_in_sand'] = pd.DataFrame(dart_plastic_in_water, columns=dart_subdatasets['plastic'].columns), pd.DataFrame(dart_plastic_in_sand, columns=dart_subdatasets['plastic'].columns)

In [19]:
dart_subdatasets['plastic_20'] = dart_subdatasets['plastic'].query("Cover_percent == 20")
dart_subdatasets['plastic_40'] = dart_subdatasets['plastic'].query("Cover_percent == 40")
dart_subdatasets['plastic_60'] = dart_subdatasets['plastic'].query("Cover_percent == 60")
dart_subdatasets['plastic_80'] = dart_subdatasets['plastic'].query("Cover_percent == 80")
dart_subdatasets['plastic_100'] = dart_subdatasets['plastic'].query("Cover_percent == 100")

dart_subdatasets['plastic_ldpe'] = dart_subdatasets['plastic'].query("Polymer == 'LDPE'")
dart_subdatasets['plastic_micronapo'] = dart_subdatasets['plastic'].query("Polymer == 'MicroNapo'")
dart_subdatasets['plastic_nylon'] = dart_subdatasets['plastic'].query("Polymer == 'Nylon'")
dart_subdatasets['plastic_pet'] = dart_subdatasets['plastic'].query("Polymer == 'PET'")
dart_subdatasets['plastic_pp'] = dart_subdatasets['plastic'].query("Polymer == 'PP'")
dart_subdatasets['plastic_pvc'] = dart_subdatasets['plastic'].query("Polymer == 'PVC'")

In [20]:
dart_nn_subdatasets = dict()
dart_nn_subdatasets['plastic'] = dataset_dart_nn.loc[dataset_dart_nn['Label'] == "Plastic"].copy()
dart_nn_subdatasets['water'] = dataset_dart_nn.loc[dataset_dart_nn['Label'] == "Water"].copy()
dart_nn_subdatasets['sand'] = dataset_dart_nn.loc[dataset_dart_nn['Label'] == "Sand"].copy()
dart_nn_subdatasets['plastic_water'] = dataset_dart_nn.query("Label == 'Water' or Label == 'Plastic'").copy()

In [21]:
dart_cubic_subdatasets = dict()
dart_cubic_subdatasets['plastic'] = dataset_dart_cubic.loc[dataset_dart_cubic['Label'] == "Plastic"].copy()
dart_cubic_subdatasets['water'] = dataset_dart_cubic.loc[dataset_dart_cubic['Label'] == "Water"].copy()
dart_cubic_subdatasets['sand'] = dataset_dart_cubic.loc[dataset_dart_cubic['Label'] == "Sand"].copy()
dart_cubic_subdatasets['plastic_water'] = dataset_dart_cubic.query("Label == 'Water' or Label == 'Plastic'").copy()#### Testing resample methods

## Observed dataset (Copernicus / USGS)

### Loading data from sand area 

In [45]:
path = str(input("Observed sand area TIFF files path: ")) #For example: files/tiff_files/coast
path, sources, dates = tiff_files.open_folders(path)

GEE exported TIFF files path: files/tiff_files/coast


In [46]:
os.chdir('../')
os.chdir('../')
os.chdir('../')

In [47]:
tiff_data = tiff_files.get_images(path, sources, dates)

In [48]:
for image in tiff_data:
    image.setAreaLabel(0, (int(image.getXSize()) - 1), 0, (image.getYSize() - 1), "Coast") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1
    for pixel in image.getPixels():
        pixel.setLabel(image.getLabelsMap())
        pixel.setCoverPercent(100)
        pixel.setPolymer("None")

In [49]:
tiff_coastal_dataset = tiff_files.build_dataset(tiff_data)

### Loading data from sea area

In [50]:
path = str(input("Observed sea area TIFF files path")) #For example: files/tiff_files/sea
path, sources, dates = tiff_files.open_folders(path) 

GEE exported TIFF files path: files/tiff_files/sea


In [51]:
os.chdir('../')
os.chdir('../')
os.chdir('../')

In [52]:
tiff_data = tiff_files.get_images(path, sources, dates) 

#### Labeling the entire area as Water

In [53]:
for image in tiff_data:
    image.setAreaLabel(0, (int(image.getXSize()) - 1), 0, (image.getYSize() - 1), "Water") #-1 é porque indice (numero das linhas e colunas) comeca em zero, enquanto len (que informa o tamanho) comeca em 1     
    for pixel in image.getPixels():
        pixel.setLabel(image.getLabelsMap())
        pixel.setCoverPercent(100)
        pixel.setPolymer("None")

#### Labeling the artificial targets as plastic and wood

In [54]:
for image in tiff_data:
    if image.getDate() == "2019_04_18":
        #A1 100% Water
        image.setPixelLabel(6, 3, "Plastic")  #A2
        image.setPixelLabel(6, 4, "Plastic")  #A3 
        image.setPixelLabel(7, 2, "Plastic")  #A4
        image.setPixelLabel(7, 3, "Plastic")  #A5 
        image.setPixelLabel(7, 4, "Plastic")  #A6
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 6 and pixel.getColumn() == 3: #A2
                pixel.setCoverPercent(30) #Bags + bottles
                pixel.setPolymer("Bags and Bottles")
            elif pixel.getLine() == 6 and pixel.getColumn() == 4: #A3
                pixel.setCoverPercent(18)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 7 and pixel.getColumn() == 2: #A4
                pixel.setCoverPercent(38)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 7 and pixel.getColumn() == 3: #A5
                pixel.setCoverPercent(33) #Bags + bottles
                pixel.setPolymer("Bags and Bottles")
            elif pixel.getLine() == 7 and pixel.getColumn() == 4: #A6
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bottles")
    elif image.getDate() == "2019_05_03":
        image.setPixelLabel(1, 15, "Plastic")  #A1
        image.setPixelLabel(1, 16, "Plastic")  #A2
        image.setPixelLabel(2, 15, "Plastic")  #A3 
        image.setPixelLabel(2, 16, "Plastic")  #A4
        image.setPixelLabel(3, 10, "Plastic")  #B1
        image.setPixelLabel(3, 11, "Plastic")  #B2 
        image.setPixelLabel(4, 11, "Plastic")  #B3
        image.setPixelLabel(5, 7, "Plastic")  #C1
        image.setPixelLabel(5, 8, "Plastic")  #C2
        image.setPixelLabel(6, 7, "Plastic")  #C3 
        image.setPixelLabel(6, 8, "Plastic")  #C4
        image.setPixelLabel(5, 12, "Plastic")  #D1
        image.setPixelLabel(5, 13, "Plastic")  #D2
        image.setPixelLabel(6, 12, "Plastic")  #D3 
        image.setPixelLabel(6, 13, "Plastic")  #D4
        image.setPixelLabel(9, 2, "Plastic")  #E1
        image.setPixelLabel(9, 3, "Plastic")  #E2
        #E3 and E4 100% Water
        image.setPixelLabel(11, 7, "Plastic")  #F1
        image.setPixelLabel(11, 8, "Plastic")  #F2
        image.setPixelLabel(11, 9, "Plastic")  #F3 
        #F4 and F5 100% Water
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 1 and pixel.getColumn() == 15: #A1
                pixel.setCoverPercent(15)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 1 and pixel.getColumn() == 16: #A2
                pixel.setCoverPercent(43)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 2 and pixel.getColumn() == 15: #A3
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 2 and pixel.getColumn() == 16: #A4
                pixel.setCoverPercent(2)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 3 and pixel.getColumn() == 10: #B1
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 3 and pixel.getColumn() == 11: #B2
                pixel.setCoverPercent(38)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 4 and pixel.getColumn() == 11: #B3
                pixel.setCoverPercent(8)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 5 and pixel.getColumn() == 7: #C1
                pixel.setCoverPercent(9)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 5 and pixel.getColumn() == 8: #C2
                pixel.setCoverPercent(5)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 6 and pixel.getColumn() == 7: #C3
                pixel.setCoverPercent(18)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 6 and pixel.getColumn() == 8: #C4
                pixel.setCoverPercent(14)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 5 and pixel.getColumn() == 12: #D1
                pixel.setCoverPercent(3)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 5 and pixel.getColumn() == 13: #D2
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 6 and pixel.getColumn() == 12: #D3
                pixel.setCoverPercent(2)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 6 and pixel.getColumn() == 13: #D4
                pixel.setCoverPercent(9)
                pixel.setPolymer("Bags") #Reeds ignored
                #elif pixel.getLine() == 6 and pixel.getColumn() == 14: #Reeds ignored
            elif pixel.getLine() == 9 and pixel.getColumn() == 2: #E1
                pixel.setCoverPercent(13)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 9 and pixel.getColumn() == 3: #E2
                pixel.setCoverPercent(27)
                pixel.setPolymer("Bottles")
            #E3 and E4 100% Water
            elif pixel.getLine() == 11 and pixel.getColumn() == 7: #F1
                pixel.setCoverPercent(10)
                pixel.setPolymer("Bottles") 
            elif pixel.getLine() == 11 and pixel.getColumn() == 8: #F2
                pixel.setCoverPercent(21)
                pixel.setPolymer("Bottles") 
            elif pixel.getLine() == 11 and pixel.getColumn() == 9: #F3
                pixel.setCoverPercent(2)
                pixel.setPolymer("Bottles") 
            #F4 and F5 100% Water
    elif image.getDate() == "2019_05_18":
        image.setPixelLabel(16, 2, "Plastic")  #A1  
        #A2 100% Water
        image.setPixelLabel(17, 2, "Plastic")  #A3
        image.setPixelLabel(17, 3, "Plastic")  #A4
        #B1 100% Water
        image.setPixelLabel(12, 5, "Plastic")  #B2
        image.setPixelLabel(13, 4, "Plastic")  #B3  
        image.setPixelLabel(13, 5, "Plastic")  #B4
        image.setPixelLabel(5, 5, "Plastic")  #C1
        image.setPixelLabel(5, 6, "Plastic")  #C2
        image.setPixelLabel(6, 5, "Plastic")  #C3
        image.setPixelLabel(6, 6, "Plastic")  #C4  
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 16 and pixel.getColumn() == 2: #A1
                pixel.setCoverPercent(17)
                pixel.setPolymer("Bags")
            #A2 100% Water  
            elif pixel.getLine() == 17 and pixel.getColumn() == 2: #A3
                pixel.setCoverPercent(27)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 17 and pixel.getColumn() == 3: #A4
                pixel.setCoverPercent(3)
                pixel.setPolymer("Bags")
            #B1 100% Water
            elif pixel.getLine() == 12 and pixel.getColumn() == 5: #B2
                pixel.setCoverPercent(2)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 13 and pixel.getColumn() == 4: #B3
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 13 and pixel.getColumn() == 5: #B4
                pixel.setCoverPercent(10)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 5 and pixel.getColumn() == 5: #C1
                pixel.setCoverPercent(5)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 5 and pixel.getColumn() == 6: #C2
                pixel.setCoverPercent(6)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 6 and pixel.getColumn() == 5: #C3
                pixel.setCoverPercent(10)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 6 and pixel.getColumn() == 6: #C4
                pixel.setCoverPercent(40)
                pixel.setPolymer("Bottles")
    elif image.getDate() == "2019_05_28":
        image.setPixelLabel(0, 8, "Plastic")  #A1  
        image.setPixelLabel(1, 8, "Plastic")  #A2  
        image.setPixelLabel(1, 9, "Plastic")  #A3
        image.setPixelLabel(4, 6, "Plastic")  #B1  
        image.setPixelLabel(5, 6, "Plastic")  #B2
        image.setPixelLabel(7, 3, "Plastic")  #C1  
        #C2 100% Water 
        image.setPixelLabel(8, 3, "Plastic")  #C3
        image.setPixelLabel(8, 4, "Plastic")  #C4  
        #C5 100% Water 
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 0 and pixel.getColumn() == 8: #A1
                pixel.setCoverPercent(7)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 1 and pixel.getColumn() == 8: #A2
                pixel.setCoverPercent(10)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 1 and pixel.getColumn() == 9: #A3
                pixel.setCoverPercent(13)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 4 and pixel.getColumn() == 6: #B1
                pixel.setCoverPercent(5)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 5 and pixel.getColumn() == 6: #B2
                pixel.setCoverPercent(8)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 7 and pixel.getColumn() == 3: #C1
                pixel.setCoverPercent(2)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 8 and pixel.getColumn() == 3: #C3
                pixel.setCoverPercent(35)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 8 and pixel.getColumn() == 4: #C4
                pixel.setCoverPercent(18)
                pixel.setPolymer("Bottles")
    elif image.getDate() == "2019_06_07":
        image.setPixelLabel(1, 4, "Plastic")  #A1  
        image.setPixelLabel(1, 5, "Plastic")  #A2  
        #image.setPixelLabel(2, 5, "Plastic")  #A3
        #image.setPixelLabel(5, 5, "Plastic")  #B1  
        #image.setPixelLabel(5, 6, "Plastic")  #B2
        #image.setPixelLabel(6, 5, "Plastic")  #B3  
        #image.setPixelLabel(6, 6, "Plastic")  #B4
        image.setPixelLabel(9, 2, "Plastic")  #C1  
        image.setPixelLabel(9, 3, "Plastic")  #C2 
        image.setPixelLabel(9, 4, "Plastic")  #C3
        image.setPixelLabel(10, 2, "Plastic")  #C4  
        image.setPixelLabel(10, 3, "Plastic")  #C5 
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 1 and pixel.getColumn() == 4: #A1
                pixel.setCoverPercent(4)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 1 and pixel.getColumn() == 5: #A2
                pixel.setCoverPercent(9)
                pixel.setPolymer("Bottles")
            #A3 100% Water 
            #B1 Reeds ignored
            #B2 Reeds ignored
            #B3 Reeds ignored
            #B4 Reeds ignored
            elif pixel.getLine() == 9 and pixel.getColumn() == 2: #C1
                pixel.setCoverPercent(3)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 9 and pixel.getColumn() == 3: #C2
                pixel.setCoverPercent(55)
                pixel.setPolymer("Bags and Bottles")
            elif pixel.getLine() == 9 and pixel.getColumn() == 4: #C3
                pixel.setCoverPercent(1)
                pixel.setPolymer("Bottles")
            elif pixel.getLine() == 10 and pixel.getColumn() == 2: #C4
                pixel.setCoverPercent(4)
                pixel.setPolymer("Bags")
            elif pixel.getLine() == 10 and pixel.getColumn() == 3: #C5
                pixel.setCoverPercent(15)
                pixel.setPolymer("Bags and Bottles")      

In [55]:
for image in tiff_data:
    if image.getDate() == "2021_06_21":
        image.setPixelLabel(3, 5, "Plastic") 
        image.setPixelLabel(3, 6, "Plastic")
        image.setPixelLabel(4, 4, "Plastic")
        image.setPixelLabel(4, 5, "Plastic") 
        image.setPixelLabel(4, 6, "Plastic")
        image.setPixelLabel(4, 7, "Plastic")
        image.setPixelLabel(5, 4, "Plastic")
        image.setPixelLabel(5, 5, "Plastic") 
        image.setPixelLabel(5, 6, "Plastic")
        image.setPixelLabel(8, 3, "Wood") 
        image.setPixelLabel(8, 4, "Wood")
        image.setPixelLabel(8, 5, "Wood")
        image.setPixelLabel(9, 3, "Wood") 
        image.setPixelLabel(9, 4, "Wood")
        image.setPixelLabel(9, 5, "Wood")
        image.setPixelLabel(9, 6, "Wood") 
        image.setPixelLabel(10, 2, "Wood") 
        image.setPixelLabel(10, 3, "Wood") 
        image.setPixelLabel(10, 4, "Wood")
        image.setPixelLabel(10, 5, "Wood")
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 3: 
                if pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 4: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 7: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 5: 
                if pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 8: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
            elif pixel.getLine() == 9: 
                if pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 10: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                elif pixel.getColumn() == 3 or pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
    elif image.getDate() == "2021_07_01":
        image.setPixelLabel(3, 3, "Plastic") 
        image.setPixelLabel(3, 4, "Plastic")
        image.setPixelLabel(3, 5, "Plastic")
        image.setPixelLabel(4, 3, "Plastic") 
        image.setPixelLabel(4, 4, "Plastic")
        image.setPixelLabel(4, 5, "Plastic")
        image.setPixelLabel(5, 3, "Plastic") 
        image.setPixelLabel(5, 4, "Plastic")
        image.setPixelLabel(5, 5, "Plastic")
        image.setPixelLabel(6, 3, "Plastic")
        image.setPixelLabel(6, 4, "Plastic") 
        image.setPixelLabel(6, 5, "Plastic")
        image.setPixelLabel(8, 2, "Wood")
        image.setPixelLabel(8, 3, "Wood")
        image.setPixelLabel(9, 1, "Wood") 
        image.setPixelLabel(9, 2, "Wood")
        image.setPixelLabel(9, 3, "Wood")
        image.setPixelLabel(9, 4, "Wood")
        image.setPixelLabel(10, 1, "Wood") 
        image.setPixelLabel(10, 2, "Wood")
        image.setPixelLabel(10, 3, "Wood")
        image.setPixelLabel(10, 4, "Wood")
        image.setPixelLabel(11, 2, "Wood") 
        image.setPixelLabel(11, 3, "Wood") 
        image.setPixelLabel(11, 4, "Wood") 
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 3: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 4: 
                if pixel.getColumn() == 3: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 4 or pixel.getColumn() == 5:
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 5: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 6: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 8: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 3: 
                    pixel.setCoverPercent(-1)
            elif pixel.getLine() == 9: 
                if pixel.getColumn() == 1: 
                    pixel.setCoverPercent(-1)
                elif pixel.getColumn() == 2 or pixel.getColumn() == 3 or pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 10: 
                if pixel.getColumn() == 1: 
                    pixel.setCoverPercent(-1)
                elif pixel.getColumn() == 2 or pixel.getColumn() == 3 or pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 11: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 3 or pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-1)
    elif image.getDate() == "2021_07_06":
        image.setPixelLabel(2, 5, "Plastic")
        image.setPixelLabel(2, 6, "Plastic")
        image.setPixelLabel(3, 4, "Plastic") 
        image.setPixelLabel(3, 5, "Plastic")
        image.setPixelLabel(3, 6, "Plastic")
        image.setPixelLabel(4, 4, "Plastic") 
        image.setPixelLabel(4, 5, "Plastic")
        image.setPixelLabel(4, 6, "Plastic")
        image.setPixelLabel(5, 4, "Plastic") 
        image.setPixelLabel(5, 5, "Plastic")
        image.setPixelLabel(5, 6, "Plastic") 
        image.setPixelLabel(7, 3, "Wood")
        image.setPixelLabel(7, 4, "Wood")
        image.setPixelLabel(8, 3, "Wood")
        image.setPixelLabel(8, 4, "Wood")
        image.setPixelLabel(8, 5, "Wood")
        image.setPixelLabel(8, 6, "Wood") 
        image.setPixelLabel(9, 3, "Wood")
        image.setPixelLabel(9, 4, "Wood")
        image.setPixelLabel(9, 5, "Wood")
        image.setPixelLabel(10, 3, "Wood")
        image.setPixelLabel(10, 4, "Wood")
        image.setPixelLabel(10, 5, "Wood")
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 2: 
                if pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 3: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 4: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 5: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 7: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
            elif pixel.getLine() == 8: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                if pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 9: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                elif pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 10: 
                if pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
    elif image.getDate() == "2021_07_21":
        image.setPixelLabel(2, 6, "Plastic")
        image.setPixelLabel(2, 7, "Plastic")
        image.setPixelLabel(2, 8, "Plastic")
        image.setPixelLabel(3, 6, "Plastic")
        image.setPixelLabel(3, 7, "Plastic")
        image.setPixelLabel(3, 8, "Plastic")
        image.setPixelLabel(4, 6, "Plastic")
        image.setPixelLabel(4, 7, "Plastic")
        image.setPixelLabel(4, 8, "Plastic")
        image.setPixelLabel(5, 6, "Plastic") 
        image.setPixelLabel(5, 7, "Plastic") 
        image.setPixelLabel(7, 4, "Wood")
        image.setPixelLabel(7, 5, "Wood")
        image.setPixelLabel(7, 6, "Wood")
        image.setPixelLabel(8, 4, "Wood")
        image.setPixelLabel(8, 5, "Wood")
        image.setPixelLabel(8, 6, "Wood")
        image.setPixelLabel(8, 7, "Wood")
        image.setPixelLabel(9, 4, "Wood")
        image.setPixelLabel(9, 5, "Wood")
        image.setPixelLabel(9, 6, "Wood")
        image.setPixelLabel(9, 7, "Wood") 
        image.setPixelLabel(10, 5, "Wood")
        image.setPixelLabel(10, 6, "Wood")
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 2: 
                if pixel.getColumn() == 6 or pixel.getColumn() == 7 or pixel.getColumn() == 8: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 3: 
                if pixel.getColumn() == 8: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 6 or pixel.getColumn() == 7: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 4: 
                if pixel.getColumn() == 6 or pixel.getColumn() == 8:  
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 7:  
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 5: 
                if pixel.getColumn() == 6 or pixel.getColumn() == 7:  
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 7: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
            elif pixel.getLine() == 8: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 7: 
                    pixel.setCoverPercent(-1)
                if pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 9: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 7: 
                    pixel.setCoverPercent(-1)
                if pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 10: 
                if pixel.getColumn() == 5 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
    elif image.getDate() == "2021_08_25": 
        image.setPixelLabel(1, 4, "Plastic") 
        image.setPixelLabel(1, 5, "Plastic") 
        image.setPixelLabel(2, 4, "Plastic")
        image.setPixelLabel(2, 5, "Plastic")
        image.setPixelLabel(2, 6, "Plastic")
        image.setPixelLabel(3, 4, "Plastic")
        image.setPixelLabel(3, 5, "Plastic")
        image.setPixelLabel(3, 6, "Plastic")
        image.setPixelLabel(4, 4, "Plastic")
        image.setPixelLabel(4, 5, "Plastic")
        image.setPixelLabel(4, 6, "Plastic")
        image.setPixelLabel(6, 3, "Wood") 
        image.setPixelLabel(7, 2, "Wood") 
        image.setPixelLabel(7, 3, "Wood") 
        image.setPixelLabel(7, 4, "Wood")
        image.setPixelLabel(7, 5, "Wood")
        image.setPixelLabel(8, 2, "Wood") 
        image.setPixelLabel(8, 3, "Wood")
        image.setPixelLabel(8, 4, "Wood")
        image.setPixelLabel(8, 5, "Wood")
        image.setPixelLabel(9, 2, "Wood") 
        image.setPixelLabel(9, 3, "Wood")
        image.setPixelLabel(9, 4, "Wood")
        image.setPixelLabel(9, 5, "Wood")
        for pixel in image.getPixels():
            pixel.setLabel(image.getLabelsMap())
            if pixel.getLine() == 1: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            if pixel.getLine() == 2: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 3: 
                if pixel.getColumn() == 6: 
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
                elif pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-100)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 4: 
                if pixel.getColumn() == 4 or pixel.getColumn() == 5 or pixel.getColumn() == 6:  
                    pixel.setCoverPercent(-1)
                    pixel.setPolymer("HDPE mesh")
            elif pixel.getLine() == 6: 
                if pixel.getColumn() == 3: 
                    pixel.setCoverPercent(-1)
            elif pixel.getLine() == 7: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 4 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                elif pixel.getColumn() == 3: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 8: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 5: 
                    pixel.setCoverPercent(-1)
                if pixel.getColumn() == 3 or pixel.getColumn() == 4: 
                    pixel.setCoverPercent(-100)
            elif pixel.getLine() == 9: 
                if pixel.getColumn() == 2 or pixel.getColumn() == 3 or pixel.getColumn() == 4 or pixel.getColumn() == 5:
                    pixel.setCoverPercent(-1)

In [56]:
tiff_marine_dataset = tiff_files.build_dataset(tiff_data)

In [59]:
tiff_dataset = pd.concat([tiff_marine_dataset, tiff_coastal_dataset], ignore_index=True)
tiff_dataset

,Path,Line,Column,Blue,Green,NIR1,NIR2,Red,RedEdge1,RedEdge2,RedEdge3,SWIR1,SWIR2,Label,Cover_percent,Polymer
0,2019_04_18,0,0,0.0408,0.0439,0.0173,0.0168,0.0223,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None
1,2019_04_18,0,1,0.0388,0.0398,0.0169,0.0168,0.0184,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None
2,2019_04_18,0,2,0.0428,0.0403,0.0166,0.0149,0.0159,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None
3,2019_04_18,0,3,0.0344,0.0305,0.0157,0.0149,0.0134,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None
4,2019_04_18,0,4,0.0294,0.0257,0.0135,0.0110,0.0116,0.0119,0.0145,0.0154,0.0035,0.0029,Water,100,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2021_08_25,9,5,0.0466,0.0733,0.1892,0.2083,0.0937,0.1093,0.1559,0.1754,0.2495,0.1615,Coast,100,None
3193,2021_08_25,9,6,0.0513,0.0754,0.1967,0.2286,0.1010,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None
3194,2021_08_25,9,7,0.0509,0.0773,0.2047,0.2286,0.1024,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None
3195,2021_08_25,9,8,0.0486,0.0713,0.2026,0.2381,0.0926,0.1229,0.1834,0.2078,0.2647,0.1822,Coast,100,None


In [60]:
tiff_dataset.to_csv(str(input("Observed files path: "))) #For example: files/csv_files/dataset_usgs.csv

Observed files path: files/csv_files/dataset_usgs.csv


### Building subdataframes

In [23]:
dataset_usgs = pd.read_csv(str(input("Observed files path: "))) #For example: files/csv_files/dataset_usgs.csv
dataset_usgs.drop('Unnamed: 0', axis=1, inplace=True)
dataset_usgs

Observed files path: files/csv_files/dataset_usgs.csv


,Path,Line,Column,Blue,Green,NIR1,NIR2,Red,RedEdge1,RedEdge2,RedEdge3,SWIR1,SWIR2,Label,Cover_percent,Polymer
0,2019_04_18,0,0,0.0408,0.0439,0.0173,0.0168,0.0223,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None
1,2019_04_18,0,1,0.0388,0.0398,0.0169,0.0168,0.0184,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None
2,2019_04_18,0,2,0.0428,0.0403,0.0166,0.0149,0.0159,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None
3,2019_04_18,0,3,0.0344,0.0305,0.0157,0.0149,0.0134,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None
4,2019_04_18,0,4,0.0294,0.0257,0.0135,0.0110,0.0116,0.0119,0.0145,0.0154,0.0035,0.0029,Water,100,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2021_08_25,9,5,0.0466,0.0733,0.1892,0.2083,0.0937,0.1093,0.1559,0.1754,0.2495,0.1615,Coast,100,None
3193,2021_08_25,9,6,0.0513,0.0754,0.1967,0.2286,0.1010,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None
3194,2021_08_25,9,7,0.0509,0.0773,0.2047,0.2286,0.1024,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None
3195,2021_08_25,9,8,0.0486,0.0713,0.2026,0.2381,0.0926,0.1229,0.1834,0.2078,0.2647,0.1822,Coast,100,None


#### Adding labels translation, year column and radiometric indices

In [24]:
usgs_label = []
for i in range(len(dataset_usgs)):
    if dataset_usgs.at[i, 'Label'] == 'Water':
        usgs_label.append('Água')
    elif dataset_usgs.at[i, 'Label'] == 'Coast':
        usgs_label.append('Costa')
    elif dataset_usgs.at[i, 'Label'] == 'Plastic':
        usgs_label.append('Plástico')
    elif dataset_usgs.at[i, 'Label'] == 'Wood':
        usgs_label.append('Madeira')

dataset_usgs['Classe'] = usgs_label

In [25]:
usgs_polymer = []

for i in range(len(dataset_usgs)):
    if dataset_usgs.at[i, 'Polymer'] == 'None':
        usgs_polymer.append('Nenhum')
    elif dataset_usgs.at[i, 'Polymer'] == 'Bags':
        usgs_polymer.append('Sacolas')
    elif dataset_usgs.at[i, 'Polymer'] == 'Bottles':
        usgs_polymer.append('Garrafas')
    elif dataset_usgs.at[i, 'Polymer'] == 'HDPE mesh':
        usgs_polymer.append('Malha de HDPE')
    elif dataset_usgs.at[i, 'Polymer'] == 'Bags and Bottles':
        usgs_polymer.append('Sacolas e garrafas')

dataset_usgs['Polímero'] = usgs_polymer
dataset_usgs

,Path,Line,Column,Blue,Green,NIR1,NIR2,Red,RedEdge1,RedEdge2,RedEdge3,SWIR1,SWIR2,Label,Cover_percent,Polymer,Classe,Polímero
0,2019_04_18,0,0,0.0408,0.0439,0.0173,0.0168,0.0223,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None,Água,Nenhum
1,2019_04_18,0,1,0.0388,0.0398,0.0169,0.0168,0.0184,0.0183,0.0181,0.0193,0.0088,0.0059,Water,100,None,Água,Nenhum
2,2019_04_18,0,2,0.0428,0.0403,0.0166,0.0149,0.0159,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None,Água,Nenhum
3,2019_04_18,0,3,0.0344,0.0305,0.0157,0.0149,0.0134,0.0130,0.0167,0.0170,0.0050,0.0040,Water,100,None,Água,Nenhum
4,2019_04_18,0,4,0.0294,0.0257,0.0135,0.0110,0.0116,0.0119,0.0145,0.0154,0.0035,0.0029,Water,100,None,Água,Nenhum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2021_08_25,9,5,0.0466,0.0733,0.1892,0.2083,0.0937,0.1093,0.1559,0.1754,0.2495,0.1615,Coast,100,None,Costa,Nenhum
3193,2021_08_25,9,6,0.0513,0.0754,0.1967,0.2286,0.1010,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None,Costa,Nenhum
3194,2021_08_25,9,7,0.0509,0.0773,0.2047,0.2286,0.1024,0.1218,0.1754,0.1965,0.2600,0.1704,Coast,100,None,Costa,Nenhum
3195,2021_08_25,9,8,0.0486,0.0713,0.2026,0.2381,0.0926,0.1229,0.1834,0.2078,0.2647,0.1822,Coast,100,None,Costa,Nenhum


In [26]:
usgs_year = []
for i in range(len(dataset_usgs)):
    if str(dataset_usgs.at[i, 'Path']).split('_')[0] == '2019':
        usgs_year.append('2019')
    elif str(dataset_usgs.at[i, 'Path']).split('_')[0] == '2021':
        usgs_year.append('2021')

dataset_usgs['Year'] = usgs_year

In [27]:
dataset_usgs['NDWI'] = (dataset_usgs['Green'] - dataset_usgs['NIR1']) / (dataset_usgs['Green'] + dataset_usgs['NIR1'])
dataset_usgs['WRI'] = (dataset_usgs['Green'] + dataset_usgs['Red']) / (dataset_usgs['NIR1'] + dataset_usgs['SWIR2'])
dataset_usgs['NDVI'] = (dataset_usgs['NIR1'] - dataset_usgs['Red']) / (dataset_usgs['NIR1'] + dataset_usgs['Red'])
dataset_usgs['AWEI'] = 4 * (dataset_usgs['Green'] - dataset_usgs['SWIR2']) - (0.25 * dataset_usgs['NIR1'] + 2.75 * dataset_usgs['SWIR1'])
dataset_usgs['MNDWI'] = (dataset_usgs['Green'] - dataset_usgs['SWIR2']) / (dataset_usgs['Green'] + dataset_usgs['SWIR2'])
dataset_usgs['SR'] = dataset_usgs['NIR1'] / dataset_usgs['Red']
dataset_usgs['PI'] = dataset_usgs['NIR1'] / (dataset_usgs['NIR1'] + dataset_usgs['Red'])
dataset_usgs['RNDVI'] = (dataset_usgs['Red'] - dataset_usgs['NIR1']) / (dataset_usgs['Red'] + dataset_usgs['NIR1'])
dataset_usgs['FDI'] = dataset_usgs['NIR1'] - (dataset_usgs['RedEdge2'] + (dataset_usgs['SWIR1'] - dataset_usgs['RedEdge2']) * ((dataset_usgs['NIR1'] - dataset_usgs['Red']) / (dataset_usgs['SWIR1'] - dataset_usgs['Red'])) * 10)

dataset_usgs

,Path,Line,Column,Blue,Green,NIR1,NIR2,Red,RedEdge1,RedEdge2,...,Year,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,2019_04_18,0,0,0.0408,0.0439,0.0173,0.0168,0.0223,0.0183,0.0181,...,2019,0.434641,2.853448,-0.126263,0.123475,0.763052,0.775785,0.436869,0.126263,0.033644
1,2019_04_18,0,1,0.0388,0.0398,0.0169,0.0168,0.0184,0.0183,0.0181,...,2019,0.403880,2.552632,-0.042493,0.107175,0.741794,0.918478,0.478754,0.042493,0.013331
2,2019_04_18,0,2,0.0428,0.0403,0.0166,0.0149,0.0159,0.0130,0.0167,...,2019,0.416520,2.728155,0.021538,0.127300,0.819413,1.044025,0.510769,-0.021538,-0.007614
3,2019_04_18,0,3,0.0344,0.0305,0.0157,0.0149,0.0134,0.0130,0.0167,...,2019,0.320346,2.228426,0.079038,0.088325,0.768116,1.171642,0.539519,-0.079038,-0.033036
4,2019_04_18,0,4,0.0294,0.0257,0.0135,0.0110,0.0116,0.0119,0.0145,...,2019,0.311224,2.274390,0.075697,0.078200,0.797203,1.163793,0.537849,-0.075697,-0.026802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2021_08_25,9,5,0.0466,0.0733,0.1892,0.2083,0.0937,0.1093,0.1559,...,2021,-0.441524,0.476190,0.337575,-1.086225,-0.375639,2.019210,0.668788,-0.337575,-0.540436
3193,2021_08_25,9,6,0.0513,0.0754,0.1967,0.2286,0.1010,0.1218,0.1754,...,2021,-0.445792,0.480523,0.321465,-1.144175,-0.386493,1.947525,0.660732,-0.321465,-0.487896
3194,2021_08_25,9,7,0.0509,0.0773,0.2047,0.2286,0.1024,0.1218,0.1754,...,2021,-0.451773,0.479072,0.333116,-1.138575,-0.375858,1.999023,0.666558,-0.333116,-0.519848
3195,2021_08_25,9,8,0.0486,0.0713,0.2026,0.2381,0.0926,0.1229,0.1834,...,2021,-0.479372,0.425936,0.372629,-1.222175,-0.437475,2.187905,0.686314,-0.372629,-0.500440


#### Cleaning data and creating subdataframes

In [28]:
dividers_usgs = dict()
dividers_usgs.update({"Green + NIR1": dataset_usgs['Green'] + dataset_usgs['NIR1']})
dividers_usgs.update({"SWIR2 + NIR1": dataset_usgs['NIR1'] + dataset_usgs['SWIR2']})
dividers_usgs.update({"Red + NIR1": dataset_usgs['NIR1'] + dataset_usgs['Red']})
dividers_usgs.update({"0.25 * NIR1 + 2.75 * SWIR1": 0.25 * dataset_usgs['NIR1'] + 2.75 * dataset_usgs['SWIR1']})
dividers_usgs.update({"Swir2 + * Green": dataset_usgs['Green'] + dataset_usgs['SWIR2']})
dividers_usgs.update({"Red": dataset_usgs['Red']})
dividers_usgs.update({"SWIR1 - Red": dataset_usgs['SWIR1'] - dataset_usgs['Red']})

zeros_usgs = dict()

for key in dividers_usgs.keys():#### Adding labels translation, year column and radiometric indices
    i = 0
    for value in dividers_usgs[key]:
        if value == 0:
            i += 1
    zeros_usgs.update({key:i})

zeros_usgs

{'Green + NIR1': 0,
 'SWIR2 + NIR1': 0,
 'Red + NIR1': 1,
 '0.25 * NIR1 + 2.75 * SWIR1': 0,
 'Swir2 + * Green': 0,
 'Red': 3,
 'SWIR1 - Red': 16}

In [29]:
query = 'NDVI > 1000 or NDVI < -1000 or SR > 1000 or SR < -1000 or PI > 1000 or PI < -1000 or RNDVI > 1000 or RNDVI < -1000 or FDI > 1000 or FDI < -1000'
indexes = dataset_usgs.query(query).index #deletando amostras com valor -inf derivados da divisâo por zero no FDI
dataset_usgs.drop(indexes,  axis=0, inplace=True) #20 pixels nâo selecionados, 1 de plástico e todos os outros de água
dataset_usgs

,Path,Line,Column,Blue,Green,NIR1,NIR2,Red,RedEdge1,RedEdge2,...,Year,NDWI,WRI,NDVI,AWEI,MNDWI,SR,PI,RNDVI,FDI
0,2019_04_18,0,0,0.0408,0.0439,0.0173,0.0168,0.0223,0.0183,0.0181,...,2019,0.434641,2.853448,-0.126263,0.123475,0.763052,0.775785,0.436869,0.126263,0.033644
1,2019_04_18,0,1,0.0388,0.0398,0.0169,0.0168,0.0184,0.0183,0.0181,...,2019,0.403880,2.552632,-0.042493,0.107175,0.741794,0.918478,0.478754,0.042493,0.013331
2,2019_04_18,0,2,0.0428,0.0403,0.0166,0.0149,0.0159,0.0130,0.0167,...,2019,0.416520,2.728155,0.021538,0.127300,0.819413,1.044025,0.510769,-0.021538,-0.007614
3,2019_04_18,0,3,0.0344,0.0305,0.0157,0.0149,0.0134,0.0130,0.0167,...,2019,0.320346,2.228426,0.079038,0.088325,0.768116,1.171642,0.539519,-0.079038,-0.033036
4,2019_04_18,0,4,0.0294,0.0257,0.0135,0.0110,0.0116,0.0119,0.0145,...,2019,0.311224,2.274390,0.075697,0.078200,0.797203,1.163793,0.537849,-0.075697,-0.026802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192,2021_08_25,9,5,0.0466,0.0733,0.1892,0.2083,0.0937,0.1093,0.1559,...,2021,-0.441524,0.476190,0.337575,-1.086225,-0.375639,2.019210,0.668788,-0.337575,-0.540436
3193,2021_08_25,9,6,0.0513,0.0754,0.1967,0.2286,0.1010,0.1218,0.1754,...,2021,-0.445792,0.480523,0.321465,-1.144175,-0.386493,1.947525,0.660732,-0.321465,-0.487896
3194,2021_08_25,9,7,0.0509,0.0773,0.2047,0.2286,0.1024,0.1218,0.1754,...,2021,-0.451773,0.479072,0.333116,-1.138575,-0.375858,1.999023,0.666558,-0.333116,-0.519848
3195,2021_08_25,9,8,0.0486,0.0713,0.2026,0.2381,0.0926,0.1229,0.1834,...,2021,-0.479372,0.425936,0.372629,-1.222175,-0.437475,2.187905,0.686314,-0.372629,-0.500440


In [30]:
usgs_subdatasets = dict()

usgs_subdatasets['plp2021'] = dataset_usgs.query('Year == "2021"').copy()
usgs_subdatasets['plp2019'] = dataset_usgs.loc[dataset_usgs['Year'] == "2019"].copy()

usgs_subdatasets['water'] = dataset_usgs.loc[dataset_usgs['Label'] == "Water"].copy()
usgs_subdatasets['coast'] = dataset_usgs.loc[dataset_usgs['Label'] == "Coast"].copy()
usgs_subdatasets['plastic'] = dataset_usgs.loc[dataset_usgs['Label'] == "Plastic"].copy()
usgs_subdatasets['wood'] = dataset_usgs.loc[dataset_usgs['Label'] == "Wood"].copy()
usgs_subdatasets['plastic_and_water'] = dataset_usgs.query("Label == 'Water' or Label == 'Plastic'").copy()

usgs_subdatasets['plp2021_plastic_water'] = usgs_subdatasets['plp2021'].query('Label == "Plastic" or Label == "Water"').copy()
usgs_subdatasets['plp2019_plastic_water'] = usgs_subdatasets['plp2019'].query('Label == "Plastic" or Label == "Water"').copy()

usgs_subdatasets['wood_-100'] = usgs_subdatasets['wood'].query('Cover_percent < -99').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['wood_-100'])):
    cover_percents.append("Unknown")
usgs_subdatasets['wood_-100']['Cover_percent'] = cover_percents

usgs_subdatasets['wood_-000'] = usgs_subdatasets['wood'].query('Cover_percent < 0 and Cover_percent > -99').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['wood_-000'])):
    cover_percents.append("Unknown")
usgs_subdatasets['wood_-000']['Cover_percent'] = cover_percents

usgs_subdatasets['wood_unknownpercent'] = usgs_subdatasets['wood'].query('Cover_percent < 100').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['wood_unknownpercent'])):
    cover_percents.append("Unknown")
usgs_subdatasets['wood_unknownpercent']['Cover_percent'] = cover_percents

usgs_subdatasets['plastic_20'] = usgs_subdatasets['plastic'].query('Cover_percent >= 0 and Cover_percent <= 20').copy()#Até 20% de cobertura plástica
usgs_subdatasets['plastic_40'] = usgs_subdatasets['plastic'].query("Cover_percent > 20 and Cover_percent <= 40").copy()#21 a 40% de cobertura plástica
usgs_subdatasets['plastic_60'] = usgs_subdatasets['plastic'].query("Cover_percent > 40 and Cover_percent <= 60").copy()#41 a 60% de cobertura plástica
usgs_subdatasets['plastic_80'] = usgs_subdatasets['plastic'].query("Cover_percent > 60 and Cover_percent <= 80").copy()#61 a 80% de cobertura plástica
usgs_subdatasets['plastic_100'] = usgs_subdatasets['plastic'].query("Cover_percent > 80").copy()#81 a 100% de cobertura plástica
usgs_subdatasets['plastic_min_20'] = usgs_subdatasets['plastic'].query("Cover_percent >= 20").copy()#min 20% de cobertura plástica 
usgs_subdatasets['plastic_min_50'] = usgs_subdatasets['plastic'].query("Cover_percent >= 50").copy()#min 50% de cobertura plástica

usgs_subdatasets['plastic_-100'] = usgs_subdatasets['plastic'].query('Cover_percent < -99').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['plastic_-100'])):
    cover_percents.append("Unknown")
usgs_subdatasets['plastic_-100']['Cover_percent'] = cover_percents

usgs_subdatasets['plastic_-000'] = usgs_subdatasets['plastic'].query('Cover_percent < 0 and Cover_percent > -99').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['plastic_-000'])):
    cover_percents.append("Unknown")
usgs_subdatasets['plastic_-000']['Cover_percent'] = cover_percents

usgs_subdatasets['plastic_unknownpercent'] = usgs_subdatasets['plastic'].query('Cover_percent < 0').copy()
cover_percents = []
for i in range(len(usgs_subdatasets['plastic_unknownpercent'])):
    cover_percents.append("Unknown")
usgs_subdatasets['plastic_unknownpercent']['Cover_percent'] = cover_percents

usgs_subdatasets['plastic_bags'] = usgs_subdatasets['plastic'].query('Polymer == "Bags"').copy()
usgs_subdatasets['plastic_bottles'] = usgs_subdatasets['plastic'].query('Polymer == "Bottles"').copy()
usgs_subdatasets['plastic_mesh'] = usgs_subdatasets['plastic'].query('Polymer == "HDPE mesh"').copy()
usgs_subdatasets['plastic_mix'] = usgs_subdatasets['plastic'].query('Polymer == "Bags and Bottles"').copy()

#### Testing resample methods (English and Portuguese)

In [33]:
datasets_names = ["Means per resampling method", "Standard deviation per resampling method"]
traces = [
            [
                [dataset_dart_nn[feature].mean() for feature in feature_names], #Nearest neighbor
                [dataset_dart[feature].mean() for feature in feature_names],#Bilinear Interpolation
                [dataset_dart_cubic[feature].mean() for feature in feature_names], #Cubic Interpolation
                [dataset_usgs[feature].mean() for feature in feature_names] #Acolite
             ],
             [
                [dataset_dart_nn[feature].std() for feature in feature_names], #Nearest neighbor
                [dataset_dart[feature].std() for feature in feature_names],#Bilinear Interpolation
                [dataset_dart_cubic[feature].std() for feature in feature_names], #Cubic Interpolation
                [dataset_usgs[feature].std() for feature in feature_names] #Acolite 
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names], [feature_names, feature_names, feature_names, feature_names]]
legends = [['DART nearest neighbor mean', 'DART bilinear interpolation mean', 'DART cubic interpolation mean', 'USGS acolite mean'],
           ['DART nearest neighbor std', 'DART bilinear interpolation std', 'DART cubic interpolation std', 'USGS acolite std']]
modes = [['markers+lines', 'markers+lines', 'markers+lines', 'markers+lines'],
         ['dash', 'dash', 'dash', 'dash']]
colors = [['#c9b207', '#008000', '#5425ff', '#FF0000'], ['#c9b207', '#008000', '#5425ff', '#FF0000']]
chart_title = "Statistics per resampling method"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1800
guidance = "horizontal"

export_name = (str(input("Path/filename: "))) 
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_std_resampling

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/english/exploratory_analysis/descriptive_statistics/mean_std_resampling


In [35]:
datasets_names = ["Médias por método de reamostragem", "Desvio padrão por método de reamostragem"]
legends = [['Média DART - vizinho mais próximo', 'Média DART - interpolação bilinear', 'Média DART - interpolação cúbica', 'Média USGS'],
           ['Std DART - vizinho mais próximo', 'Std DART - interpolação bilinear', 'Std DART - interpolação cúbica', 'Std USGS']]
chart_title = "Estatísticas por método de reamostragem"
x_title = "Banda"
y_title = "Reflectância"

export_name = (str(input("Path/filename: "))) 
#Por exemplo: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_reamostragem

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_reamostragem


In [36]:
datasets_names = ['DART nearest neighbor quartiles', 'DART bilinear interpolation quartiles', 'DART cubic interpolation quartiles', 'USGS quartiles']

traces = [
            [
                [dataset_dart_nn[feature].describe()['min'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dataset_dart[feature].describe()['min'] for feature in feature_names],
                [dataset_dart[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dataset_dart_cubic[feature].describe()['min'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['max'] for feature in feature_names]
             ],
            [
                [dataset_usgs[feature].describe()['min'] for feature in feature_names],
                [dataset_usgs[feature].describe()['25%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['50%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['75%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names],
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['DART nearest neighbor min', 'DART nearest neighbor 25%', 'DART nearest neighbor 50%', 'DART nearest neighbor 75%', 'DART nearest neighbor max'],
            ['DART bilinear interpolation min', 'DART bilinear interpolation 25%', 'DART bilinear interpolation 50%', 'DART bilinear interpolation 75%', 'DART bilinear interpolation max'],
            ['DART cubic interpolation min', 'DART cubic interpolation 25%', 'DART cubic interpolation 50%', 'DART cubic interpolation 75%', 'DART cubic interpolation max'],
            ['USGS min', 'USGS 25%', 'USGS 50%', 'USGS 75%', 'USGS max']
          ]

modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#f9e54e', '#f9e54e', '#d9c108', '#a89506', '#a89506'], 
          ['#00b300', '#00b300', '#009100', '#005e00', '#005e00'],
          ['#7c58ff', '#7c58ff', '#3903ff', '#2900be', '#2900be'],
          ['#f44', '#f44', '#d00', '#b00', '#b00']]

chart_title = "Statistics per resampling method"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1800
guidance = "horizontal"

export_name = (str(input("Path/filename: "))) 
#Por exemplo: charts/english/exploratory_analysis/descriptive_statistics/quartiles_resampling_method

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/english/exploratory_analysis/descriptive_statistics/quartiles_resampling_method


In [37]:
datasets_names = ['Quartis DART - vizinho mais próximo', 'Quartis DART - interpolação bilinear', 'Quartis DART - interpolação cúbica', 'Quartis USGS']
legends = [
            ['DART vizinho mais próximo mín', 'DART vizinho mais próximo 25%', 'DART vizinho mais próximo 50%', 'DART vizinho mais próximo 75%', 'DART vizinho mais próximo máx'],
            ['DART interpolação bilinear mín', 'DART interpolação bilinear 25%', 'DART interpolação bilinear 50%', 'DART interpolação bilinear 75%', 'DART interpolação bilinear máx'],
            ['DART interpolação cúbica mín', 'DART interpolação cúbica 25%', 'DART interpolação cúbica 50%', 'DART interpolação cúbica 75%', 'DART interpolação cúbica máx'],
            ['USGS mín', 'USGS 25%', 'USGS 50%', 'USGS 75%', 'USGS máx']
          ]

chart_title = "Estatísticas por método de reamostragem"
x_title = "Banda"
y_title = "Reflectância"
height = 600
width = 1900
guidance = "horizontal"

export_name = (str(input("Path/filename: "))) 
#Por exemplo: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_metodo_reamostragem

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_metodo_reamostragem


In [38]:
datasets_names = ['DART quartiles', 'USGS quartiles']

traces = [
            [
                [dataset_dart_nn[feature].describe()['min'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart_nn[feature].describe()['max'] for feature in feature_names],
                [dataset_dart[feature].describe()['min'] for feature in feature_names],
                [dataset_dart[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart[feature].describe()['max'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['min'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart_cubic[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dataset_usgs[feature].describe()['min'] for feature in feature_names],
                [dataset_usgs[feature].describe()['25%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['50%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['75%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names, 
           feature_names, feature_names, feature_names, feature_names, feature_names, 
           feature_names, feature_names, feature_names, feature_names, feature_names],
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['DART nearest neighbor min', 'DART nearest neighbor 25%', 'DART nearest neighbor 50%', 'DART nearest neighbor 75%', 'DART nearest neighbor max',
             'DART bilinear interpolation min', 'DART bilinear interpolation 25%', 'DART bilinear interpolation 50%', 'DART bilinear interpolation 75%', 'DART bilinear interpolation max',
             'DART cubic interpolation min', 'DART cubic interpolation 25%', 'DART cubic interpolation 50%', 'DART cubic interpolation 75%', 'DART cubic interpolation max'],
            ['USGS min', 'USGS 25%', 'USGS 50%', 'USGS 75%', 'USGS max']
          ]

modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines',
          'dot', 'dash', 'dash', 'dash', 'markers+lines',
          'dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#f9e54e', '#f9e54e', '#d9c108', '#a89506', '#a89506', 
           '#00b300', '#00b300', '#009100', '#005e00', '#005e00',
           '#7c58ff', '#7c58ff', '#3903ff', '#2900be', '#2900be'],
          ['#f44', '#f44', '#d00', '#b00', '#b00']]

chart_title = "Statistics per resampling method"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1200
guidance = "horizontal"

export_name = (str(input("Path/filename: "))) 
#Por exemplo: charts/english/exploratory_analysis/descriptive_statistics/quartiles_resampling_method_grouped

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/english/exploratory_analysis/descriptive_statistics/quartiles_resampling_method_grouped


In [39]:
datasets_names = ['Quartis DART', 'Quartis USGS']

legends = [
            ['DART vizinho mais próximo mín', 'DART vizinho mais próximo 25%', 'DART vizinho mais próximo 50%', 'DART vizinho mais próximo 75%', 'DART vizinho mais próximo máx',
             'DART interpolação bilinear mín', 'DART interpolação bilinear 25%', 'DART interpolação bilinear 50%', 'DART interpolação bilinear 75%', 'DART interpolação bilinear máx',
             'DART interpolação cúbica mín', 'DART interpolação cúbica 25%', 'DART interpolação cúbica 50%', 'DART interpolação cúbica 75%', 'DART interpolação cúbica máx'],
            ['USGS mín', 'USGS 25%', 'USGS 50%', 'USGS 75%', 'USGS máx']
          ]

chart_title = "Estatísticas por método de reamostragem"
x_title = "Banda"
y_title = "Reflectância"

export_name = (str(input("Path/filename: "))) 
#Por exemplo: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_metodo_reamostragem_agrupado

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Path/filename: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_metodo_reamostragem_agrupado


## Exploratory analysis

### Kolmogorov-Smirnov

In [40]:
print("DART x USGS")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dataset_dart[feature], dataset_usgs[feature]))
print("*******************")
print("                   ")

print("DART (only Plastic and water) x USGS (only Plastic and water)")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['plastic_and_water'][feature], usgs_subdatasets['plastic_and_water'][feature]))
print("*******************")
print("                   ")

print("DART (only Plastic and water) x USGS 2019 (only Plastic and water)")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['plastic_and_water'][feature], usgs_subdatasets['plp2019_plastic_water'][feature]))
print("*******************")
print("                   ")

print("DART (only Plastic and water) x USGS 2021 (only Plastic and water)")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['plastic_and_water'][feature], usgs_subdatasets['plp2021_plastic_water'][feature]))
print("*******************")
print("                   ")


print("USGS 2019 x USGS 2021")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(usgs_subdatasets['plp2019'][feature], usgs_subdatasets['plp2021'][feature]))
print("*******************")
print("                   ")

DART x USGS
Blue KstestResult(statistic=0.504449718184515, pvalue=0.0)
Green KstestResult(statistic=0.5038564224265796, pvalue=0.0)
Red KstestResult(statistic=0.5035416600721572, pvalue=0.0)
RedEdge1 KstestResult(statistic=0.5005932957579353, pvalue=0.0)
RedEdge2 KstestResult(statistic=0.5005932957579353, pvalue=0.0)
RedEdge3 KstestResult(statistic=0.5005932957579353, pvalue=0.0)
NIR1 KstestResult(statistic=0.5005932957579353, pvalue=0.0)
NIR2 KstestResult(statistic=0.5005932957579353, pvalue=0.0)
SWIR1 KstestResult(statistic=0.49581070074354316, pvalue=0.0)
SWIR2 KstestResult(statistic=0.45729983948147035, pvalue=0.0)
NDWI KstestResult(statistic=0.41655739622903304, pvalue=0.0)
WRI KstestResult(statistic=0.5818937577610956, pvalue=0.0)
NDVI KstestResult(statistic=0.8415228036501976, pvalue=0.0)
AWEI KstestResult(statistic=0.5027258784049962, pvalue=0.0)
MNDWI KstestResult(statistic=0.49564098258143785, pvalue=0.0)
SR KstestResult(statistic=0.836486605979439, pvalue=0.0)
PI KstestResul

In [41]:
print("DART Plastic x USGS Plastic")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['plastic'][feature], usgs_subdatasets['plastic'][feature]))
print("*******************")
print("                   ")

print("DART Water x USGS Water")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['water'][feature], usgs_subdatasets['water'][feature]))
print("*******************")
print("                   ")

print("DART Sand x USGS Coast")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['sand'][feature], usgs_subdatasets['coast'][feature]))
print("*******************")
print("                   ")

print("USGS Wood x USGS Coast")
for feature in feature_names + radiometric_indexes:
    print(feature, ks_2samp(dart_subdatasets['sand'][feature], usgs_subdatasets['wood'][feature]))
print("*******************")
print("                   ")

DART Plastic x USGS Plastic
Blue KstestResult(statistic=0.856957928802589, pvalue=1.9653712833371636e-85)
Green KstestResult(statistic=0.8736245954692556, pvalue=7.650820404681667e-91)
Red KstestResult(statistic=0.8736245954692556, pvalue=7.650820404681667e-91)
RedEdge1 KstestResult(statistic=0.7, pvalue=2.5610651237603943e-50)
RedEdge2 KstestResult(statistic=0.6, pvalue=1.8159615559465044e-35)
RedEdge3 KstestResult(statistic=0.6, pvalue=1.8159615559465044e-35)
NIR1 KstestResult(statistic=0.8736245954692556, pvalue=7.650820404681667e-91)
NIR2 KstestResult(statistic=0.6, pvalue=1.8159615559465044e-35)
SWIR1 KstestResult(statistic=0.6891585760517799, pvalue=1.65289490221338e-48)
SWIR2 KstestResult(statistic=0.6058252427184466, pvalue=3.1414986300068715e-36)
NDWI KstestResult(statistic=0.5323624595469255, pvalue=2.1448938952158855e-27)
WRI KstestResult(statistic=0.39239482200647247, pvalue=1.317450066417033e-14)
NDVI KstestResult(statistic=0.5601941747572815, pvalue=1.5040220820948129e-30

### Descriptive statistics

In [42]:
rsdata_charts.pie_chart(
    [
    pd.concat([dart_subdatasets['plastic'], dart_subdatasets['sand'], dart_subdatasets['water']], ignore_index=True),
    pd.concat([usgs_subdatasets['plp2019'].query('Label=="Plastic"'), usgs_subdatasets['plp2019'].query('Label=="Coast"'), usgs_subdatasets['plp2019'].query('Label=="Water"')], ignore_index=True),
    pd.concat([usgs_subdatasets['plp2021'].query('Label=="Plastic"'), usgs_subdatasets['plp2021'].query('Label=="Coast"'), usgs_subdatasets['plp2021'].query('Label=="Water"'), usgs_subdatasets['plp2021'].query('Label=="Wood"')], ignore_index=True)
    ], 
    ['Label', 'Label', 'Label'], 
    ["DART (simulation)","USGS 2019", "USGS 2021"], 
    "DART x USGS classes", 630, 900, 
    ['#FF69B4', '#FFD700', '#1E90FF'], str(input("Chart path: ")))
    #For example: charts/english/exploratory_analysis/descriptive_statistics/classes

Chart path: charts/english/exploratory_analysis/descriptive_statistics/classes


In [43]:
rsdata_charts.pie_chart(
    [
    pd.concat([dart_subdatasets['plastic'], dart_subdatasets['sand'], dart_subdatasets['water']], ignore_index=True),
    pd.concat([usgs_subdatasets['plp2019'].query('Label=="Plastic"'), usgs_subdatasets['plp2019'].query('Label=="Coast"'), usgs_subdatasets['plp2019'].query('Label=="Water"')], ignore_index=True),
    pd.concat([usgs_subdatasets['plp2021'].query('Label=="Plastic"'), usgs_subdatasets['plp2021'].query('Label=="Coast"'), usgs_subdatasets['plp2021'].query('Label=="Water"'), usgs_subdatasets['plp2021'].query('Label=="Wood"')], ignore_index=True)
    ], 
    ['Classe', 'Classe', 'Classe'], 
    ["DART (simulação)", "USGS 2019", "USGS 2021"], 
    "Classes DART x USGS", 630, 900, 
    ['#FF69B4', '#FFD700', '#1E90FF'], str(input("Caminho do gráfico: ")))
    #For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/classes

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/classes


In [44]:
rsdata_charts.pie_chart([dart_subdatasets['plastic']], 
                        ['Polymer'], ["Polymers in DART data (simulated)"], " ", 600, 450, 
                        ['#ADF224','#1AB1B1', '#2f1b70', '#D81F88', '#FF8825', '#F1C800'], 
                        str(input("Chart path: ")))
                        #For example: charts/english/exploratory_analysis/descriptive_statistics/polymers_dart

Chart path: charts/english/exploratory_analysis/descriptive_statistics/polymers_dart


In [45]:
rsdata_charts.pie_chart([dart_subdatasets['plastic']], 
                        ['Polymer'], ["Polímeros no dados DART (simulados)"], " ", 600, 450, 
                        ['#ADF224','#1AB1B1', '#2f1b70', '#D81F88', '#FF8825', '#F1C800'], 
                        str(input("Caminho do gráfico: ")))
                        #For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/polimeros_dart


Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/polimeros_dart


In [46]:
rsdata_charts.pie_chart([pd.concat([usgs_subdatasets['plp2019'].query('Polymer!="None"'), usgs_subdatasets['plp2021'].query('Polymer!="None"')], ignore_index=True)], 
                        ['Polymer'], ["Polymers in USGS data (real)"], " ", 600, 450, 
                        ['#49C658','#8945AB', '#FF675F', '#FCFE5E'], 
                        str(input("Chart path: ")))
                        #For example: charts/english/exploratory_analysis/descriptive_statistics/polymers_usgs

Chart path: charts/english/exploratory_analysis/descriptive_statistics/polymers_usgs


In [47]:
rsdata_charts.pie_chart([pd.concat([usgs_subdatasets['plp2019'].query('Polymer!="None"'), usgs_subdatasets['plp2021'].query('Polymer!="None"')], ignore_index=True)], 
                        ['Polímero'], ["Polímeros nos dados USGS (reais)"], " ", 600, 450, 
                        ['#49C658', '#8945AB', '#FF675F', '#FCFE5E'], 
                        str(input("Caminho do gráfico: ")))
                        #For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/polimeros_usgs

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/polimeros_usgs


In [48]:
rsdata_charts.pie_chart([dataset_usgs], ['Year'], [" "], "USGS data sources", 400, 400, ['#c20', '#ff8d77'], str(input("Chart path: ")))
#For example: charts/english/exploratory_analysis/descriptive_statistics/usgs_sources

Chart path: charts/english/exploratory_analysis/descriptive_statistics/usgs_sources


In [49]:
rsdata_charts.pie_chart([dataset_usgs], ['Year'], [" "], "Fontes dos dados USGS", 400, 400, ['#c20', '#ff8d77'], str(input("Caminho do gráfico: ")))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/fontes_usgs

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/fontes_usgs


In [50]:
rsdata_charts.pie_chart([usgs_subdatasets['plp2019'], usgs_subdatasets['plp2021']], ['Path', 'Path'], 
                        ["Days in 2019", "Days in 2021"], "USGS acquisition dates", 500, 1000, ['#991900', '#c20', '#f53', '#ff9c88', '#ffc6bb'], 
                        str(input("Chart path: ")))
                        #For example: charts/english/exploratory_analysis/descriptive_statistics/usgs_dates

Chart path: charts/english/exploratory_analysis/descriptive_statistics/usgs_dates


In [51]:
rsdata_charts.pie_chart([usgs_subdatasets['plp2019'], usgs_subdatasets['plp2021']], ['Path', 'Path'], 
                        ["Dias em 2019", "Dias em 2021"], "Datas de aquisição USGS", 500, 1000, ['#991900', '#c20', '#f53', '#ff9c88', '#ffc6bb'], 
                        str(input("Caminho do gráfico: ")))
                        #For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/datas_usgs

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/datas_usgs


In [52]:
rsdata_charts.pie_chart([usgs_subdatasets['plastic'], usgs_subdatasets['water'], usgs_subdatasets['coast'], usgs_subdatasets['wood']], ['Year', 'Year', 'Year', 'Year'], 
                        ["Plastic", "Water", "Coast", "Wood"], "USGS data sources - per class", 400, 1200, ['#c20', '#ff8d77'], 
                        str(input("Chart path: ")))
                        #For example: charts/english/exploratory_analysis/descriptive_statistics/usgs_sources_per_class

Chart path: charts/english/exploratory_analysis/descriptive_statistics/usgs_sources_per_class


In [53]:
rsdata_charts.pie_chart([usgs_subdatasets['plastic'], usgs_subdatasets['water'], usgs_subdatasets['coast'], usgs_subdatasets['wood']], ['Year', 'Year', 'Year', 'Year'], 
                        ["Plástico", "Água", "Costa", "Madeira"], "Fontes dos dados USGS - por classe", 400, 1200, ['#c20', '#ff8d77'], 
                        str(input("Caminho do gráfico: ")))
                        #For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/fontes_usgs_por_classe


Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/fontes_usgs_por_classe


In [54]:
datasets_names = ["Mean","Std"]
traces = [
            [
                [dataset_dart[feature].mean() for feature in feature_names],
                [dataset_usgs[feature].mean() for feature in feature_names]
             ],
             [
                [dataset_dart[feature].std() for feature in feature_names],
                [dataset_usgs[feature].std() for feature in feature_names]
             ]
          ]
labels = [[feature_names, feature_names], [feature_names, feature_names]]
legends = [['DART mean', 'USGS mean'],
           ['DART std', 'USGS std']]
modes = [['markers+lines', 'markers+lines'],
         ['dash', 'dash']]
colors = [['#008000', '#FF0000'], ['#008000', '#FF0000']]
chart_title = "DART x USGS statistics"
x_title = "Band"
y_title = "Reflectance"
height = 450
width = 950
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_std

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/mean_std


In [55]:
datasets_names = ["Média","Desvio padrão"]
legends = [['Média DART', 'Média USGS'],
           ['Desvio padrão DART', 'Desvio padrão USGS']]
chart_title = "Estatísticas DART x USGS"
x_title = "Banda"
y_title = "Reflectância"
width = 1050

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao


In [56]:
datasets_names = ["Mean (only plastic and water)","Std (only plastic and water)"]
traces = [
            [
                [dart_subdatasets['plastic_and_water'][feature].mean() for feature in feature_names], 
                [usgs_subdatasets['plastic_and_water'][feature].mean() for feature in feature_names]
             ],
             [
                [dart_subdatasets['plastic_and_water'][feature].std() for feature in feature_names], 
                [usgs_subdatasets['plastic_and_water'][feature].std() for feature in feature_names]
             ]
          ]
labels = [[feature_names, feature_names], [feature_names, feature_names]]
legends = [['DART mean (only plastic and water)', 'USGS mean (only plastic and water)'],
           ['DART std (only plastic and water)', 'USGS std (only plastic and water)']]
modes = [['markers+lines', 'markers+lines'],
         ['dash', 'dash']]
colors = [['#008000', '#FF0000'], ['#008000', '#FF0000']]
chart_title = "DART x USGS statistics (only plastic and water)"
x_title = "Band"
y_title = "Reflectance"
height = 450
width = 950
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_std_plastic_water

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/mean_std_plastic_water


In [57]:
datasets_names = ["Média (apenas plástico e água)","Desvio padrão (apenas plástico e água)"]
legends = [['Média DART', 'Média USGS'],
           ['Desvio padrão DART', 'Desvio padrão USGS']]
chart_title = "Estatísticas DART x USGS (apenas plástico e água)"
x_title = "Banda"
y_title = "Reflectância"
width = 1050

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_plastico_agua

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_plastico_agua


In [58]:
datasets_names = ["Mean (only plastic and water)","Std (only plastic and water)"]
traces = [
            [
                [dart_subdatasets['plastic_and_water'][feature].mean() for feature in feature_names], 
                [usgs_subdatasets['plp2019_plastic_water'][feature].mean() for feature in feature_names],
                [usgs_subdatasets['plp2021_plastic_water'][feature].mean() for feature in feature_names]
             ],
             [
                [dart_subdatasets['plastic_and_water'][feature].std() for feature in feature_names], 
                [usgs_subdatasets['plp2019_plastic_water'][feature].std() for feature in feature_names],
                [usgs_subdatasets['plp2021_plastic_water'][feature].std() for feature in feature_names],
             ]
          ]
labels = [[feature_names, feature_names, feature_names], [feature_names, feature_names, feature_names]]
legends = [['DART mean (only plastic and water)', 'USGS 2019 mean (only plastic and water)', 'USGS 2021 mean (only plastic and water)'],
           ['DART std (only plastic and water)', 'USGS 2019 std (only plastic and water)', 'USGS 2021 std (only plastic and water)']]
modes = [['markers+lines', 'markers+lines', 'markers+lines'],
         ['dash', 'dash', 'dash']]
colors = [['#008000', '#FF0000', '#00D'], ['#008000', '#FF0000', '#00D']]
chart_title = "DART x USGS statistics (only plastic and water)"
x_title = "Band"
y_title = "Reflectance"
height = 450
width = 950
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_std_plastic_water_per_year

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/mean_std_plastic_water_per_year


In [59]:
datasets_names = ["Média (apenas plástico e água)","Desvio padrão (apenas plástico e água)"]
legends = [['Média DART', 'Média USGS 2019', 'Média USGS 2021'],
           ['Desvio padrão DART', 'Desvio padrão USGS 2019', 'Desvio padrão USGS 2021']]
chart_title = "Estatísticas DART x USGS (apenas plástico e água)"
x_title = "Banda"
y_title = "Reflectância"
width = 1050

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_plastico_agua_por ano

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/media_dpadrao_plastico_agua_por ano


In [60]:
datasets_names = ["DART (simulated data)", "USGS (real data)"]

traces = [
             [
                [dataset_dart[feature].describe()['min'] for feature in feature_names],
                [dataset_dart[feature].describe()['25%'] for feature in feature_names],
                [dataset_dart[feature].describe()['50%'] for feature in feature_names],
                [dataset_dart[feature].describe()['75%'] for feature in feature_names],
                [dataset_dart[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dataset_usgs[feature].describe()['min'] for feature in feature_names],
                [dataset_usgs[feature].describe()['25%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['50%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['75%'] for feature in feature_names],
                [dataset_usgs[feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names],
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [['DART min', 'DART 25%', 'DART 50%', 'DART 75%', 'DART max'],
           ['USGS min', 'USGS 25%', 'USGS 50%', 'USGS 75%', 'USGS max']]

modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#008000', '#008000', '#008000', '#008000', '#008000'],
          ['#FF0000', '#FF0000', '#FF0000', '#FF0000', '#FF0000']]

chart_title = "DART x USGS quartiles"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1400
guidance = "horizontal"


export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/quartiles

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/quartiles


In [61]:
datasets_names = ["DART (dados simulados)", "USGS (dados reais)"]

chart_title = "Quartis DART x USGS"
x_title = "Banda"
y_title = "Reflectância"
guidance = "horizontal"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis


In [62]:
datasets_names = ["DART Plastic (simulated)", "DART Water (simulated)", "DART Sand (simulated)"]

traces = [
            [
                [dart_subdatasets['plastic'][feature].describe()['min'] for feature in feature_names],
                [dart_subdatasets['plastic'][feature].describe()['25%'] for feature in feature_names],
                [dart_subdatasets['plastic'][feature].describe()['50%'] for feature in feature_names],
                [dart_subdatasets['plastic'][feature].describe()['75%'] for feature in feature_names],
                [dart_subdatasets['plastic'][feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dart_subdatasets['water'][feature].describe()['min'] for feature in feature_names],
                [dart_subdatasets['water'][feature].describe()['25%'] for feature in feature_names],
                [dart_subdatasets['water'][feature].describe()['50%'] for feature in feature_names],
                [dart_subdatasets['water'][feature].describe()['75%'] for feature in feature_names],
                [dart_subdatasets['water'][feature].describe()['max'] for feature in feature_names]
             ],
             [
                [dart_subdatasets['sand'][feature].describe()['min'] for feature in feature_names],
                [dart_subdatasets['sand'][feature].describe()['25%'] for feature in feature_names],
                [dart_subdatasets['sand'][feature].describe()['50%'] for feature in feature_names],
                [dart_subdatasets['sand'][feature].describe()['75%'] for feature in feature_names],
                [dart_subdatasets['sand'][feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['DART Plastic min', 'DART Plastic 25%', 'DART Plastic 50%', 'DART Plastic 75%', 'DART Plastic max'],
            ['DART Water min', 'DART Water 25%', 'DART Water 50%', 'DART Water 75%', 'DART Water max'],
            ['DART Sand min', 'DART Sand 25%', 'DART Sand 50%', 'DART Sand 75%', 'DART Sand max']
          ]

modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#ffadd6', '#ffadd6', '#FF69B4', '#ff148a', '#ff148a'], 
          ['#73baff', '#73baff', '#1E90FF', '#005db7', '#005db7'],
          ['#ffe766', '#ffe766', '#FFD700', '#ddba00', '#ddba00']]

chart_title = "Class statistics - DART quartiles"
x_title = "Band"
y_title = "Reflectance"
height = 500
width = 1200
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_dart

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_dart


In [63]:
datasets_names = ["DART Plástico (simulado)", "DART Água (simulado)", "DART Areia (simulado)"]

legends = [
            ['DART Plástico mín', 'DART Plástico 25%', 'DART Plástico 50%', 'DART Plástico 75%', 'DART Plástico máx'],
            ['DART Água mín', 'DART Água 25%', 'DART Água 50%', 'DART Água 75%', 'DART Água máx'],
            ['DART Areia mín', 'DART Areia 25%', 'DART Areia 50%', 'DART Areia 75%', 'DART Areia máx']
          ]

chart_title = "Estatísticas por classe - Quartis DART"
x_title = "Banda"
y_title = "Reflectância"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_dart

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_dart


In [64]:
datasets_names = ["USGS Plastic (real)", "USGS Water (real)", "USGS Coast (real)", "USGS Wood (real)"]

traces = [
             [
                [usgs_subdatasets['plastic'][feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['plastic'][feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['plastic'][feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['plastic'][feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['plastic'][feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['water'][feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['water'][feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['water'][feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['water'][feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['water'][feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['coast'][feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['coast'][feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['coast'][feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['coast'][feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['coast'][feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['wood'][feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['wood'][feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['wood'][feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['wood'][feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['wood'][feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['USGS Plastic min', 'USGS Plastic 25%', 'USGS Plastic 50%', 'USGS Plastic 75%', 'USGS Plastic max'],
            ['USGS Water min', 'USGS Water 25%', 'USGS Water 50%', 'USGS Water 75%', 'USGS Water max'],
            ['USGS Coast min', 'USGS Coast 25%', 'USGS Coast 50%', 'USGS Coast 75%', 'USGS Coast max'],
            ['USGS Wood min', 'USGS Wood 25%', 'USGS Wood 50%', 'USGS Wood 75%', 'USGS Wood max']]


modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#ffadd6', '#ffadd6', '#FF69B4', '#ff148a', '#ff148a'], 
          ['#73baff', '#73baff', '#1E90FF', '#005db7', '#005db7'],
          ['#ffe766', '#ffe766', '#FFD700', '#ddba00', '#ddba00'],
          ['#a38fd3', '#a38fd3', '#7152bb', '#533990', '#533990']
         ]

chart_title = "Class statistics - USGS quartiles"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1200
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs


In [65]:
datasets_names = ["USGS Plástico (real)", "USGS Água (real)", "USGS Costa (real)", "USGS Madeira (real)"]

legends = [
            ['USGS Plástico mín', 'USGS Plástico 25%', 'USGS Plástico 50%', 'USGS Plástico 75%', 'USGS Plástico máx'],
            ['USGS Água mín', 'USGS Water 25%', 'USGS Água 50%', 'USGS Água 75%', 'USGS Água máx'],
            ['USGS Costa mín', 'USGS Costa 25%', 'USGS Costa 50%', 'USGS Costa 75%', 'USGS Costa máx'],
            ['USGS Madeira mín', 'USGS Madeira 25%', 'USGS Madeira 50%', 'USGS Madeira 75%', 'USGS Madeira máx']]

chart_title = "Estatísticas por classe - Quartis USGS"
x_title = "Banda"
y_title = "Reflectância"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs


In [66]:
datasets_names = ["USGS 2019 Plastic (real)", "USGS 2019 Water (real)", "USGS 2019 Coast (real)"]

traces = [
             [
                [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['water'].query('Year == "2019"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2019"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2019"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2019"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2019"')[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['coast'].query('Year == "2019"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2019"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2019"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2019"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2019"')[feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['USGS Plastic min', 'USGS Plastic 25%', 'USGS Plastic 50%', 'USGS Plastic 75%', 'USGS Plastic max'],
            ['USGS Water min', 'USGS Water 25%', 'USGS Water 50%', 'USGS Water 75%', 'USGS Water max'],
            ['USGS Coast min', 'USGS Coast 25%', 'USGS Coast 50%', 'USGS Coast 75%', 'USGS Coast max']
          ]


modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']]

colors = [['#ffadd6', '#ffadd6', '#FF69B4', '#ff148a', '#ff148a'], 
          ['#73baff', '#73baff', '#1E90FF', '#005db7', '#005db7'],
          ['#ffe766', '#ffe766', '#FFD700', '#ddba00', '#ddba00']]

chart_title = "Class statistics - USGS 2019 quartiles"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1200
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs_2019

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs_2019


In [67]:
datasets_names = ["USGS 2019 Plástico (real)", "USGS 2019 Água (real)", "USGS 2019 Costa (real)"]

legends = [
            ['USGS Plástico mín', 'USGS Plástico 25%', 'USGS Plástico 50%', 'USGS Plástico 75%', 'USGS Plástico máx'],
            ['USGS Água mín', 'USGS Water 25%', 'USGS Água 50%', 'USGS Água 75%', 'USGS Água máx'],
            ['USGS Costa mín', 'USGS Costa 25%', 'USGS Costa 50%', 'USGS Costa 75%', 'USGS Costa máx']]

chart_title = "Estatísticas por classe - Quartis USGS 2019"
x_title = "Banda"
y_title = "Reflectância"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs_2019

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs_2019


In [68]:
datasets_names = ["USGS 2021 Plastic (real)", "USGS 2021 Water (real)", "USGS 2021 Coast (real)", "USGS 2021 Wood (real)"]

traces = [
             [
                [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['water'].query('Year == "2021"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2021"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2021"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2021"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['water'].query('Year == "2021"')[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['coast'].query('Year == "2021"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2021"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2021"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2021"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['coast'].query('Year == "2021"')[feature].describe()['max'] for feature in feature_names]
             ],
             [
                [usgs_subdatasets['wood'].query('Year == "2021"')[feature].describe()['min'] for feature in feature_names],
                [usgs_subdatasets['wood'].query('Year == "2021"')[feature].describe()['25%'] for feature in feature_names],
                [usgs_subdatasets['wood'].query('Year == "2021"')[feature].describe()['50%'] for feature in feature_names],
                [usgs_subdatasets['wood'].query('Year == "2021"')[feature].describe()['75%'] for feature in feature_names],
                [usgs_subdatasets['wood'].query('Year == "2021"')[feature].describe()['max'] for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names, feature_names]]

legends = [
            ['USGS Plastic min', 'USGS Plastic 25%', 'USGS Plastic 50%', 'USGS Plastic 75%', 'USGS Plastic max'],
            ['USGS Water min', 'USGS Water 25%', 'USGS Water 50%', 'USGS Water 75%', 'USGS Water max'],
            ['USGS Coast min', 'USGS Coast 25%', 'USGS Coast 50%', 'USGS Coast 75%', 'USGS Coast max'],
            ['USGS Wood min', 'USGS Wood 25%', 'USGS Wood 50%', 'USGS Wood 75%', 'USGS Wood max']]


modes = [['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines'],
         ['dot', 'dash', 'dash', 'dash', 'markers+lines']
        ]

colors = [['#ffadd6', '#ffadd6', '#FF69B4', '#ff148a', '#ff148a'], 
          ['#73baff', '#73baff', '#1E90FF', '#005db7', '#005db7'],
          ['#ffe766', '#ffe766', '#FFD700', '#ddba00', '#ddba00'],
          ['#a38fd3', '#a38fd3', '#7152bb', '#533990', '#533990']
         ]

chart_title = "Class statistics - USGS 2021 quartiles"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1200
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs_2021

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/class_quartiles_usgs_2021


In [69]:
datasets_names = ["USGS 2021 Plástico (real)", "USGS 2021 Água (real)", "USGS 2021 Costa (real)", "USGS 2021 Madeira (real)"]

legends = [
            ['USGS Plástico mín', 'USGS Plástico 25%', 'USGS Plástico 50%', 'USGS Plástico 75%', 'USGS Plástico máx'],
            ['USGS Água mín', 'USGS Water 25%', 'USGS Água 50%', 'USGS Água 75%', 'USGS Água máx'],
            ['USGS Costa mín', 'USGS Costa 25%', 'USGS Costa 50%', 'USGS Costa 75%', 'USGS Costa máx'],
            ['USGS Madeira mín', 'USGS Madeira 25%', 'USGS Madeira 50%', 'USGS Madeira 75%', 'USGS Madeira máx']]

chart_title = "Estatísticas por classe - Quartis USGS 2021"
x_title = "Banda"
y_title = "Reflectância"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs_2021

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/quartis_por_classe_usgs_2021


In [70]:
datasets_names = ["DART", "USGS", "USGS (minimum plastic: 20%)", "USGS (minimum plastic: 50%)"]

traces = [
             [
                 [dart_subdatasets['plastic'][feature].mean() for feature in feature_names],
                 [dart_subdatasets['water'][feature].mean() for feature in feature_names],
                 [dart_subdatasets['sand'][feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic'][feature].mean() for feature in feature_names],
                 [usgs_subdatasets['water'][feature].mean() for feature in feature_names],
                 [usgs_subdatasets['coast'][feature].mean() for feature in feature_names],
                 [usgs_subdatasets['wood'][feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic_min_20'][feature].mean() for feature in feature_names], 
                 [usgs_subdatasets['water'][feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic_min_50'][feature].mean() for feature in feature_names],
                 [usgs_subdatasets['water'][feature].mean() for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names], 
          [feature_names, feature_names]]

legends = [['Plastic (DART mean)', 'Water (DART mean)', 'Sand (DART mean)'],
           ['Plastic (USGS mean)', 'Water (USGS mean)', 'Coast (USGS mean)', 'Wood (USGS mean)'],
           ['Plastic - min 20% (USGS mean)', 'Water (USGS mean)'],
           ['Plastic - min 50% (USGS mean)', 'Water (USGS mean)']]

modes = [['lines', 'lines', 'lines'],
         ['dot', 'dot', 'dot', 'dot'],
         ['dash', 'dash'],
         ['markers+lines', 'markers+lines']]

colors = [['#FF69B4', '#1E90FF', '#FFD700'], 
          ['#FF69B4', '#1E90FF', '#FFD700', '#7152bb'],
          ['#FF69B4', '#1E90FF'],
          ['#FF69B4', '#1E90FF']]

chart_title = "Class statistics - Mean spectral signatures"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1300
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_spectral_signatures

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/mean_spectral_signatures


In [71]:
datasets_names = ["DART", "USGS", "USGS (cobertura plástica mínima: 20%)", "USGS (cobertura plástica mínima: 50%)"]

legends = [['Plástico (média DART)', 'Água (média DART)', 'Areia (média DART)'],
           ['Plástico (média USGS)', 'Água (média USGS)', 'Costa (média USGS)', 'Madeira (média USGS)'],
           ['Plástico - min 20% (média USGS)', 'Água (média USGS)'],
           ['Plástico - min 50% (média USGS)', 'Água (média USGS)']]

chart_title = "Estatísticas por classe - Assinaturas espectrais médias"
x_title = "Banda"
y_title = "Reflectância"
width = 1550

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/assinaturas_espectrais_medias

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/assinaturas_espectrais_medias


In [72]:
datasets_names = ["DART", "USGS 2019", "USGS 2021", "USGS (minimum plastic: 50%)"]

traces = [
             [
                 [dart_subdatasets['plastic'][feature].mean() for feature in feature_names],
                 [dart_subdatasets['water'][feature].mean() for feature in feature_names],
                 [dart_subdatasets['sand'][feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic'].query('Year == "2019"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['water'].query('Year == "2019"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['coast'].query('Year == "2019"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['wood'].query('Year == "2019"')[feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic'].query('Year == "2021"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['water'].query('Year == "2021"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['coast'].query('Year == "2021"')[feature].mean() for feature in feature_names],
                 [usgs_subdatasets['wood'].query('Year == "2021"')[feature].mean() for feature in feature_names]
             ],
             [
                 [usgs_subdatasets['plastic_min_50'][feature].mean() for feature in feature_names],
                 [usgs_subdatasets['water'][feature].mean() for feature in feature_names]
             ]
          ]

labels = [[feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names, feature_names, feature_names], 
          [feature_names, feature_names]]

legends = [['Plastic (DART mean)', 'Water (DART mean)', 'Sand (DART mean)'],
           ['Plastic (USGS 2019 mean)', 'Water (USGS 2019 mean)', 'Coast (USGS 2019 mean)', 'Wood (USGS 2019 mean)'],
           ['Plastic (USGS 2021 mean)', 'Water (USGS 2021 mean)', 'Coast (USGS 2021 mean)', 'Wood (USGS 2021 mean)'],
           ['Plastic - min 50% (USGS mean)', 'Water (USGS mean)']]

modes = [['lines', 'lines', 'lines'],
         ['dot', 'dot', 'dot', 'dot'],
         ['dash', 'dash', 'dash', 'dash'],
         ['markers+lines', 'markers+lines']]

colors = [['#FF69B4', '#1E90FF', '#FFD700'], 
          ['#FF69B4', '#1E90FF', '#FFD700', '#7152bb'],
          ['#FF69B4', '#1E90FF', '#FFD700', '#7152bb'],
          ['#FF69B4', '#1E90FF']]

chart_title = "Class statistics - Mean spectral signatures"
x_title = "Band"
y_title = "Reflectance"
height = 600
width = 1300
guidance = "horizontal"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/descriptive_statistics/mean_spectral_signatures_per_year

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/descriptive_statistics/mean_spectral_signatures_per_year


In [73]:
datasets_names = ["DART", "USGS 2019", "USGS 2021", "USGS (cobertura plástica mínima: 50%)"]

legends = [['Plástico (média DART)', 'Água (média DART)', 'Areia (média DART)'],
           ['Plástico (média USGS 2019)', 'Água (média USGS 2019)', 'Costa (média USGS 2019)', 'Madeira (média USGS 2019)'],
           ['Plástico (média USGS 2021)', 'Água (média USGS 2021)', 'Costa (média USGS 2021)', 'Madeira (média USGS 2021)'],
           ['Plástico - min 50% (média USGS)', 'Água (média USGS)']]

chart_title = "Estatísticas por classe - Assinaturas espectrais médias"
x_title = "Banda"
y_title = "Reflectância"
width = 1550

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/estatisticas_descritivas/assinaturas_espectrais_medias_por_ano

rsdata_charts.line_chart(datasets_names, traces, labels, legends, modes, colors, chart_title, x_title, y_title, height, width, legend_orientation = "v", guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/estatisticas_descritivas/assinaturas_espectrais_medias_por_ano


In [76]:
datasets_names = ["DART features correlation", "USGS features correlation"]

datasets = [round(dataset_dart[feature_names + radiometric_indexes].corr(),3), round(dataset_usgs[feature_names + radiometric_indexes].corr(),3)]

x_labels = [feature_names + radiometric_indexes, feature_names + radiometric_indexes]

y_labels = [feature_names + radiometric_indexes, feature_names + radiometric_indexes]

colorscale = "RdBu"

chart_title = "Features correlation"

height = 1600

width = 1000

guidance = "vertical"

export_name = str(input("Chart path: "))
#For example: charts/english/exploratory_analysis/correlation/features_correlation

rsdata_charts.heatmap_chart(datasets_names, datasets, x_labels, y_labels, colorscale, chart_title, height, width,  guidance=guidance, export_name=export_name)

Chart path: charts/english/exploratory_analysis/correlation/features_correlation


In [78]:
datasets_names = ["Dados DART", "Dados USGS"]

chart_title = "Correlação"

guidance = "vertical"

export_name = str(input("Caminho do gráfico: "))
#For example: charts/portugues/analise_exploratoria/correlacao/correlacao

rsdata_charts.heatmap_chart(datasets_names, datasets, x_labels, y_labels, colorscale, chart_title, height, width,  guidance=guidance, export_name=export_name)

Caminho do gráfico: charts/portugues/analise_exploratoria/correlacao/correlacao


In [79]:
rsdata_charts.heatmap_chart(["DART plastic features correlation"], 
                                round(dart_subdatasets['plastic'][feature_names + radiometric_indexes].corr(),3), 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_name = str(input("Dart plastic correlation chart path: "))
                                #For example: charts/english/exploratory_analysis/correlation/dart_plastic
                                )

rsdata_charts.heatmap_chart(["DART water features correlation"], 
                                round(dart_subdatasets['water'][feature_names + radiometric_indexes].corr(),3),
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical",
                                export_name = str(input("Dart water correlation chart path: "))
                                #For example: files/charts/english/exploratory_analysis/correlation/dart_water
                                )

rsdata_charts.heatmap_chart(["DART sand features correlation"], 
                                round(dart_subdatasets['sand'][feature_names + radiometric_indexes].corr(),3), 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes,
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_name = str(input("Dart sand correlation chart path: "))
                                #For example: charts/english/exploratory_analysis/correlation/dart_sand
                                )

Dart plastic correlation chart path: charts/english/exploratory_analysis/correlation/dart_plastic
Dart water correlation chart path: charts/english/exploratory_analysis/correlation/dart_water
Dart sand correlation chart path: charts/english/exploratory_analysis/correlation/dart_sand


In [80]:
rsdata_charts.heatmap_chart(["DART - Correlação no plástico"], 
                                round(dart_subdatasets['plastic'][feature_names + radiometric_indexes].corr(),3), 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_name = str(input("Local para o gráfico de correlação no plástico simulado: "))
                                #For example: charts/portugues/analise_exploratoria/correlacao/dart_plastico
                                )

rsdata_charts.heatmap_chart(["DART - Correlação na água"], 
                                round(dart_subdatasets['water'][feature_names + radiometric_indexes].corr(),3),
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical",
                                export_name = str(input("Local para o gráfico de correlação da água simulada: "))
                                #For example: charts/portugues/analise_exploratoria/correlacao/dart_agua
                                )

rsdata_charts.heatmap_chart(["DART - Correlação na areia"], 
                                round(dart_subdatasets['sand'][feature_names + radiometric_indexes].corr(),3), 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_name = str(input("Local para o gráfico de correlação da areia simulada: "))
                                #For example: files/charts/portugues/analise_exploratoria/correlacao/dart_areia
                                )

Local para o gráfico de correlação no plástico simulado: charts/portugues/analise_exploratoria/correlacao/dart_plastico
Local para o gráfico de correlação da água simulada: charts/portugues/analise_exploratoria/correlacao/dart_agua
Local para o gráfico de correlação da areia simulada: charts/portugues/analise_exploratoria/correlacao/dart_areia


In [81]:
datasets_names = [["USGS plastic features correlation"], ["USGS water features correlation"], 
                  ["USGS coast features correlation"], ["DART wood features correlation"]]

datasets = [round(usgs_subdatasets['plastic'][feature_names + radiometric_indexes].corr(),3),
            round(usgs_subdatasets['water'][feature_names + radiometric_indexes].corr(),3),
            round(usgs_subdatasets['coast'][feature_names + radiometric_indexes].corr(),3),
            round(usgs_subdatasets['wood'][feature_names + radiometric_indexes].corr(),3)]

export_names = [str(input("USGS plastic correlation chart path: ")), 
               str(input("USGS water correlation chart path: ")), 
               str(input("USGS coast correlation chart path: ")), 
               str(input("USGS wood correlation chart path: "))] 
                                
            #For example: charts/english/exploratory_analysis/correlation/usgs_plastic,
            #charts/english/exploratory_analysis/correlation/usgs_water,
            #charts/english/exploratory_analysis/correlation/usgs_coast,
            #charts/english/exploratory_analysis/correlation/usgs_wood]

for i in range(len(datasets_names)):
    rsdata_charts.heatmap_chart(datasets_names[i], 
                                datasets[i], 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_names[i])

USGS plastic correlation chart path: charts/english/exploratory_analysis/correlation/usgs_plastic
USGS water correlation chart path: charts/english/exploratory_analysis/correlation/usgs_water
USGS coast correlation chart path: charts/english/exploratory_analysis/correlation/usgs_coast
USGS wood correlation chart path: charts/english/exploratory_analysis/correlation/usgs_wood


In [82]:
datasets_names = [["USGS - Correlação no plástico"], ["USGS - Correlação na água"], 
                  ["USGS - Correlação na costa"], ["USGS - Correlação na madeira"]]

export_names = [str(input("Local do gráfico de correlação de plástico observado: ")), 
               str(input("Local do gráfico de correlação de água observada: ")), 
               str(input("Local do gráfico de correlação de costa observada: ")), 
               str(input("Local do gráfico de correlação de madeira observada: "))] 
                                
            #For example: charts/portugues/analise_exploratoria/correlacao/usgs_plastico,
            #charts/portugues/analise_exploratoria/correlacao/usgs_agua,
            #charts/portugues/analise_exploratoria/correlacao/usgs_costa,
            #charts/portugues/analise_exploratoria/correlacao/usgs_madeira

for i in range(len(datasets_names)):
    rsdata_charts.heatmap_chart(datasets_names[i], 
                                datasets[i], 
                                feature_names + radiometric_indexes, 
                                feature_names + radiometric_indexes, 
                                "RdBu", 
                                " ", 
                                900, 
                                900, 
                                "vertical", 
                                export_names[i])

Local do gráfico de correlação de plástico observado: charts/portugues/analise_exploratoria/correlacao/usgs_plastico
Local do gráfico de correlação de água observada: charts/portugues/analise_exploratoria/correlacao/usgs_agua
Local do gráfico de correlação de costa observada: charts/portugues/analise_exploratoria/correlacao/usgs_costa
Local do gráfico de correlação de madeira observada: charts/portugues/analise_exploratoria/correlacao/usgs_madeira


In [84]:
datasets_names = ["DART", "USGS"]

traces = [
            [dart_subdatasets['plastic'], dart_subdatasets['water'], dart_subdatasets['sand']],
            [usgs_subdatasets['plastic'], usgs_subdatasets['water'], usgs_subdatasets['coast'], usgs_subdatasets['wood']]
         ]
labels = [
            ["Plastic", "Water", "Sand"],
            ["Plastic", "Water", "Coast", "Wood"]
         ]

labels_group = [
            "DART classes", 
            "USGS classes"
        ]

n_bins = 8

colors = [
            ['#ffadd6', '#73baff', '#ffe766'], 
            ['#ffadd6', '#73baff', '#ffe766', '#a38fd3'], 
         ]

y_title = "Reflectance"
height = 450
width = 700

path = str(input("Charts path: "))
#For example: charts/english/exploratory_analysis/histograms/

for feature in feature_names + radiometric_indexes:
    bands = [feature, feature]
    
    chart_title = "Frequency percent distribution in "+feature
    x_title = feature    
    export_name = path+feature
    

    rsdata_charts.stackedbars_chart(datasets_names, traces, bands, n_bins, labels, labels_group, colors, chart_title, x_title, y_title, height, width, export_name)  

Charts path: charts/english/exploratory_analysis/histograms/


In [86]:
labels = [
            ["Plástico", "Água", "Areia"],
            ["Plástico", "Água", "Costa", "Madeira"]
         ]

labels_group = [
            "Classes DART", 
            "Classes USGS"
        ]

y_title = "Reflectância"

path = str(input("Local dos gráficos: "))
#For example: charts/portugues/analise_exploratoria/histogramas/

for feature in feature_names + radiometric_indexes:
    bands = [feature, feature]
    
    chart_title = "Distribuição percentual de frequências no atributo "+feature
    x_title = feature    
    export_name = path+feature

    rsdata_charts.stackedbars_chart(datasets_names, traces, bands, n_bins, labels, labels_group, colors, chart_title, x_title, y_title, height, width, export_name)  

Local dos gráficos: charts/portugues/analise_exploratoria/histogramas/


In [89]:
datasets_names = ["DART classes", "USGS classes"]

labels = [
            ["Sand", "Water", "Plastic"],    
            ["Coast", "Water", "Plastic", "Wood"]
         ]

colors = [
            ['#ffe766', '#73baff', '#ffadd6'], 
            ['#ffe766', '#73baff', '#ffadd6', '#a38fd3'], 
         ]

y_title = "Values"
height = 600
width = 950
guidance="horizontal"

path = str(input("Dart boxplots charts path: ")) 
#For example: charts/english/exploratory_analysis/boxplots/classes_
                                

for feature in ["NDVI", "FDI"]:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic'][feature]
                ],
                [
                    usgs_subdatasets['coast'][feature],
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic'][feature],
                    usgs_subdatasets['wood'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = feature+" index scattering"

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

Dart boxplots charts path: charts/english/exploratory_analysis/boxplots/classes_


In [90]:
datasets_names = ["Classes DART", "Classes USGS"]

labels = [
            ["Areia", "Água", "Plástico"],    
            ["Costa", "Água", "Plástico", "Madeira"]
         ]

y_title = "Valores"
height = 600
width = 950
guidance="horizontal"

path = str(input("Caminhos para gráficos boxplot dos dados simulados (DART): ")) 
#For example: charts/portugues/analise_exploratoria/boxplots/classes_

for feature in ["NDVI", "FDI"]:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic'][feature]
                ],
                [
                    usgs_subdatasets['coast'][feature],
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic'][feature],
                    usgs_subdatasets['wood'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = "Espalhamento no índice "+feature

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

Caminhos para gráficos boxplot dos dados simulados (DART): charts/portugues/analise_exploratoria/boxplots/classes_


In [91]:
datasets_names = ["DART Plastic in Sand", "DART Plastic in Water", "USGS Plastic in Water"]

labels = [
            ["Sand", "20% plastic", "40% plastic", "60% plastic", "80% plastic", "100% plastic"],    
            ["Water", "20% plastic", "40% plastic", "60% plastic", "80% plastic", "100% plastic"],
            ["Water", "20% plastic", "40% plastic", "60% plastic", "80% plastic", "100% plastic", "Unknown percent"]
         ]

colors = [
            ['#ffe766', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6'],            
            ['#73baff', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6'],
            ['#73baff', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6']            
         ]


y_title = "Reflectance"
height = 600
width = 950
guidance="horizontal"

path = str(input("DART plastic boxplots charts path: ")) 
#For example: charts/english/exploratory_analysis/boxplots/

for feature in feature_names:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 100")[feature]
                ],
                [
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 100")[feature]
                ],
                [
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic_20'][feature],
                    usgs_subdatasets['plastic_40'][feature],
                    usgs_subdatasets['plastic_60'][feature],
                    usgs_subdatasets['plastic_80'][feature],
                    usgs_subdatasets['plastic_100'][feature],
                    usgs_subdatasets['plastic_unknownpercent'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = feature+" band scattering"

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

Dart plastic boxplots charts path: charts/english/exploratory_analysis/boxplots/


In [92]:
y_title = "Values"

for feature in radiometric_indexes:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 100")[feature]
                ],
                [
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 100")[feature]
                ],
                [
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic_20'][feature],
                    usgs_subdatasets['plastic_40'][feature],
                    usgs_subdatasets['plastic_60'][feature],
                    usgs_subdatasets['plastic_80'][feature],
                    usgs_subdatasets['plastic_100'][feature],
                    usgs_subdatasets['plastic_unknownpercent'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = feature+" index scattering"

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

In [93]:
datasets_names = ["DART - Plástico na areia", "DART - Plástico na água", "USGS - Plástico na água"]

labels = [
            ["Areia", "20% plástico", "40% plástico", "60% plástico", "80% plástico", "100% plástico"],    
            ["Água", "20% plástico", "40% plástico", "60% plástico", "80% plástico", "100% plástico"],
            ["Água", "20% plástico", "40% plástico", "60% plástico", "80% plástico", "100% plástico", "Percentual desconhecido"]
         ]

colors = [
            ['#ffe766', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6'],            
            ['#73baff', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6'],
            ['#73baff', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6', '#ffadd6']            
         ]


y_title = "Reflectância"
height = 600
width = 950
guidance="horizontal"

path = str(input("Caminhos para gráficos boxplot do plástico simulado (DART):")) 
#For example: charts/portugues/analise_exploratoria/boxplots/

for feature in feature_names:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 100")[feature]
                ],
                [
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 100")[feature]
                ],
                [
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic_20'][feature],
                    usgs_subdatasets['plastic_40'][feature],
                    usgs_subdatasets['plastic_60'][feature],
                    usgs_subdatasets['plastic_80'][feature],
                    usgs_subdatasets['plastic_100'][feature],
                    usgs_subdatasets['plastic_unknownpercent'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = "Espalhamento na banda "+feature

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

Caminhos para gráficos boxplot do plástico simulado (DART):charts/portugues/analise_exploratoria/boxplots/


In [94]:
y_title = "Valores"

for feature in radiometric_indexes:
    traces = [
                [
                    dart_subdatasets['sand'][feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_sand'].query("Cover_percent == 100")[feature]
                ],
                [
                    dart_subdatasets['water'][feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 20")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 40")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 60")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 80")[feature],
                    dart_subdatasets['plastic_in_water'].query("Cover_percent == 100")[feature]
                ],
                [
                    usgs_subdatasets['water'][feature],
                    usgs_subdatasets['plastic_20'][feature],
                    usgs_subdatasets['plastic_40'][feature],
                    usgs_subdatasets['plastic_60'][feature],
                    usgs_subdatasets['plastic_80'][feature],
                    usgs_subdatasets['plastic_100'][feature],
                    usgs_subdatasets['plastic_unknownpercent'][feature]
                ]
        ]

    x_title = feature
    
    chart_title = "Espalhamento no índice "+feature

    export_name = path+feature

    rsdata_charts.boxplot_chart(datasets_names, traces, labels, colors, chart_title, x_title, y_title, height, width, guidance, export_name=export_name)

In [100]:
datasets_names = ["DART", "USGS"]

traces = [
            [dart_subdatasets['plastic'], dart_subdatasets['water'], dart_subdatasets['sand']],
            [usgs_subdatasets['plastic'], usgs_subdatasets['water'], usgs_subdatasets['coast'], usgs_subdatasets['wood']]
         ]
    
labels = [
            ["Plastic", "Water", "Sand"],
            ["Plastic", "Water", "Coast", "Wood"]
         ]

labels_group = [
                "DART classes", 
                "USGS classes"
                ]


colors = [
            ['#ffadd6', '#73baff', '#ffe766'], 
            ['#ffadd6', '#73baff', '#ffe766', '#a38fd3'], 
         ]


chart_title = "Classes scattering"
height = 400
width = 900

x_title = "Pixel Ids"
path = str(input("DART scatter charts path: ")) 
#For example: charts/english/exploratory_analysis/scatter/

for feature in feature_names + radiometric_indexes:
    y = feature
    y_title = feature
    
    export_name = path+feature+"/"+y

    rsdata_charts.scatter_chart_und(datasets_names, traces, y, labels, labels_group, colors, chart_title, x_title, y_title, height, width, legend_orientation="v", guidance="horizontal", export_name=export_name) 

DART scatter charts path: charts/english/exploratory_analysis/scatter/


In [101]:
labels = [
            ["Plástico", "Água", "Areia"],
            ["Plástico", "Água", "Costa", "Madeira"]
         ]

labels_group = [
                "Classes DART", 
                "Classes USGS"
                ]

chart_title = "Dispersão das classes"
height = 400
width = 900

path = str(input("Caminhos para gráficos de dispersão dos dados simulados (DART): ")) 
#For example: charts/portugues/analise_exploratoria/dispersao/

for feature in feature_names + radiometric_indexes:
    y = feature
    y_title = feature
    
    export_name = path+feature+"/"+y

    rsdata_charts.scatter_chart_und(datasets_names, traces, y, labels, labels_group, colors, chart_title, x_title, y_title, height, width, legend_orientation="v", guidance="horizontal", export_name=export_name) 

Caminhos para gráficos de dispersão dos dados simulados (DART):charts/portugues/analise_exploratoria/dispersao/


In [ ]:
datasets_names = ["DART", "USGS"]

traces = [
            [dart_subdatasets['plastic'], dart_subdatasets['water'], dart_subdatasets['sand']],
            [usgs_subdatasets['plastic'], usgs_subdatasets['water'], usgs_subdatasets['coast'], usgs_subdatasets['wood']]
         ]
    
labels = [
            ["Plastic", "Water", "Sand"],
            ["Plastic", "Water", "Coast", "Wood"]
         ]

labels_group = [
                "DART classes", 
                "USGS classes"
                ]


colors = [
            ['#ffadd6', '#73baff', '#ffe766'], 
            ['#ffadd6', '#73baff', '#ffe766', '#a38fd3'], 
         ]


chart_title = "Classes scattering"
height = 400
width = 900

path = str(input("DART scatter charts path: ")) 
#For example: charts/english/exploratory_analysis/scatter/

for feature_a in feature_names + radiometric_indexes:
    x = feature_a
    x_title = feature_a
    for feature_b in feature_names + radiometric_indexes:
        y = feature_b
        y_title = feature_b
        export_name = path+feature_a+"/"+x+"_x_"+y

        rsdata_charts.scatter_chart(datasets_names, traces, x, y, labels, labels_group, colors, chart_title, x_title, y_title, height, width, legend_orientation="v", guidance="horizontal", export_name=export_name) 

DART scatter charts path: charts/english/exploratory_analysis/scatter/


In [ ]:
labels = [
            ["Plástico", "Água", "Areia"],
            ["Plástico", "Água", "Costa", "Madeira"]
         ]

labels_group = [
                "Classes DART", 
                "Classes USGS"
                ]

chart_title = "Dispersão das classes"
height = 400
width = 900

path = str(input("Caminhos para gráficos de espalhamento dos dados simulados (DART):")) 
#For example: charts/portugues/analise_exploratoria/dispersao/

for feature_a in feature_names + radiometric_indexes:
    x = feature_a
    x_title = feature_a
    for feature_b in feature_names + radiometric_indexes:
        y = feature_b
        y_title = feature_b
        export_name = path+feature_a+"/"+x+"_x_"+y

        rsdata_charts.scatter_chart(datasets_names, traces, x, y, labels, labels_group, colors, chart_title, x_title, y_title, height, width, legend_orientation="v", guidance="horizontal", export_name=export_name) 

In [ ]:
#1) Revisar e postar código atual
#2) Adaptar o código para ler o banco novo (fork)
#2.a) Nao esquecer de ler a banda NIRa
#3) Repetir as análises anteriores sem a banda artificial
#4) Adicionar o Miss Forest e comparar as bandas artificiais nas imagens simuladas e observadas #https://betterdatascience.com/python-missforest-algorithm/
#5) Classificação
#5.a) Otimizar os hiper parâmetros com espaço de busca menor (somente funcoes relu e tanh, camadas menor)
#5.b) Desenvolver a análise espacial
#5.c) Procurar insights para proposta de indice
